In [ ]:
!pip install rpy2==3.5.1

Lo primero que tenemos que hacer es cargar las librerías a utilizar.

In [ ]:
import pandas as pd
import numpy as np
!pip install yfinance
import yfinance as yf
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from functools import reduce
from numpy.ma.core import append

In [3]:
# Link del repositorio de  Gitgub de la Tarea 1
%load_ext rpy2.ipython

!rm -rf RiesgosFinancieros
!git clone "https://github.com/JuanPabloAS11/Tarea1-AdRF"

Cloning into 'Tarea1-AdRF'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 23 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (23/23), 226.65 KiB | 4.63 MiB/s, done.


1. Calcular el VaR y CVaR a un día por los métodos Simulación Histórica con y sin Alisado. Tomar como día de valoración el 10 de marzo de 2023.

# Acciones
100 acciones de GCarso, -5000 de América Móvil y 1200 de Walmart. Todas de la BMV

Primero vamos a definir algunas funciones que nos serán de utilidad en pasos posteriores


In [4]:
# Función que calcula el VaR condicional
def CVaR(x , VaR):
  return np.mean(x[x >= VaR])

In [5]:
def ExponentialWeighted(primer, num_esc):
  return [primer*((1-primer)**i) for i in range(num_esc)]

In [6]:
def VaR_CA(x,P,alpha):
  aux = pd.DataFrame({"X":x, "Proba":P})
  aux = aux.sort_values(by = ["X"])
  aux["Acumulado"] = aux["Proba"].cumsum()
  aux2 = aux[aux["Acumulado"]>=(alpha)]
  return aux2["X"].values[0]

In [7]:
def CVaR_CA(x,P,alpha):
  aux = pd.DataFrame({"X":x, "Proba":P})
  aux = aux.sort_values(by = ["X"])
  aux["Acumulado"] = aux["Proba"].cumsum()
  plist = aux["Acumulado"]/aux["Acumulado"].values[len(aux)-1]
  aux2 = aux[plist>alpha]
  loss = aux2["X"].values
  esc = aux2["Proba"].values
  return sum(loss*esc)/sum(esc)

Ahora creamos una clase, donde se incluyen funciones para el cálculo del VaR y CVaR por Simulación Histórica

In [8]:
class Riesgo_Equity_FX:

  def __init__(self):
    self.posiciones = None
    self.acciones = None
    self.VaR = None
    self.CVaR = None
    self.alpha = None
    self.summary = None
    self.alisado = None
  
  def compute(self,acciones, posiciones, fecha_valoracion, alpha = 0.98 , alisado = False):
    stock_data = yf.download(acciones, end = fecha_valoracion)
    X=stock_data.Close
    X = X.dropna()
    X = X[::-1]
    X0 = X.head(1)
    DeltaX = (X.shift(1)/X)-1
    DeltaX.iloc[0,:] = 0
    X_s = X0.values*(1+DeltaX)
    posiciones = pd.Series(posiciones,index = acciones)
    PL = (X_s-X0.values[0])*posiciones
    PL['Total']=PL.sum(axis=1)
    VaR_C=[]

    if alisado:
      c = 1
      PL.dropna(inplace = True)
      #PL = PL.iloc[::-1]
      PL["Esc_SA"] = ExponentialWeighted(0.05,PL.shape[0])
      col_aux = list(PL.columns)
      col_aux.remove("Esc_SA")
      vector_VaR = [VaR_CA(PL.iloc[:,i],PL.Esc_SA,alpha) for i in range(PL.shape[1]-1)]
      VaR = pd.Series(vector_VaR,index = col_aux)
    else:
      c = 0
      vector_VaR = [PL.iloc[:,i].quantile(1-alpha) for i in range(len(acciones))]
      VaR = pd.Series(vector_VaR,index = acciones)
      VaR = PL.quantile(alpha,numeric_only = True)

    
    for i in range(0,PL.shape[1]-c):
      VC=CVaR(PL.iloc[:,i],VaR[i])
      VaR_C.append(VC)

    self.summary = pd.DataFrame({"VaR":VaR})
    self.summary['CVaR'] = VaR_C

    self.posiciones = posiciones
    self.acciones = acciones
    self.VaR = self.summary[['VaR']]
    self.CVaR = self.summary[['CVaR']]
    self.alpha = alpha

    return


Ahora utilizamos la función compute, para cargar los datos de las Acciones indicadas

##  Sin Alisado

Para este caso vamos a utilizar la función compute que está definida en la clase anterior, y vamos a especificar el parámetro alisado=False

In [9]:
misAcciones = Riesgo_Equity_FX()

In [10]:
# En los valores de los parámetros introducimos la clave de la Acción que se encuentra en Yahoo Finance, el número
# de acciones a valorar, la fecha de valoración al 10 de marzo de 2023, un valor de alpha del 98%, y especificamos
# el parámetro alisado=False.
misAcciones.compute(['GCARSOA1.MX','AMXL.MX','WALMEX.MX'], [1000, -5000, 1200], '2023-03-10', alpha = 0.98,alisado=False)

[*********************100%***********************]  3 of 3 completed


Realizamos un resumen de los valores, utilizando la función Summary incluida en la clase Riesgo_Equity_FX.
En este resumen podemos visualizar el VaR y CVaR de las tres acciones, así como el total.

In [11]:
misAcciones.summary

,VaR,CVaR
AMXL.MX,3204.055424,4719.718225
GCARSOA1.MX,4357.661394,7036.790563
WALMEX.MX,3140.064853,4234.112958
Total,5321.514639,7636.094292


In [12]:
# Alternativa
fecha_valoracion = '2023-03-10'
acciones = ['GCARSOA1.MX','AMXL.MX','WALMEX.MX']
posiciones = [1000, -5000, 1200]
alpha = 0.98

stock_data = yf.download(acciones, end = fecha_valoracion)
X=stock_data.Close
X = X.dropna()
X = X[::-1]
X0 = X.head(1)
DeltaX = (X.shift(1)/X)-1
DeltaX.iloc[0,:] = 0
X_s = X0.values*(1+DeltaX)
posiciones = pd.Series(posiciones,index = acciones)
PL = (X_s-X0.values[0])*posiciones
PL['Total']=PL.sum(axis=1)
vector_VaR = [PL.iloc[:,i].quantile(1-alpha) for i in range(len(acciones))]
vector_VaR
VaR = pd.Series(vector_VaR,index = acciones)
VaR = PL.quantile(alpha,numeric_only = True)
VaR

[*********************100%***********************]  3 of 3 completed


AMXL.MX        3204.055424
GCARSOA1.MX    4357.661394
WALMEX.MX      3140.064853
Total          5321.514639
Name: 0.98, dtype: float64

Por lo tanto llegamos a los mismos resultados

In [13]:
# Visualizamos el valor de cada acción en la Fecha de Valoración (Notemos que nos arroja el 
# día 09 de marzo)

X.head(1)

,AMXL.MX,GCARSOA1.MX,WALMEX.MX
Date,,,
2023-03-09,19.51,87.400002,72.040001


In [15]:
# Multiplicamos por el número de posiciones para obneter su valor.
X0*posiciones

,AMXL.MX,GCARSOA1.MX,WALMEX.MX
Date,,,
2023-03-09,-97550.001144,87400.001526,86448.001099


## Con Alisado

Ahora para el caso Sin Alisado, de igual manera utilizamos la función compute, sólo que especificando en el parámetro alisado=True

In [16]:
misAcciones.compute(['GCARSOA1.MX','AMXL.MX','WALMEX.MX'], [1000, -5000, 1200], '2023-03-10', alpha = 0.98, alisado=True)

[*********************100%***********************]  3 of 3 completed


Y realizamos un resumen, en donde se muestra el VaR y CVaR

In [17]:
misAcciones.summary

,VaR,CVaR
AMXL.MX,2829.693230,4251.101000
GCARSOA1.MX,2847.859373,4731.781394
WALMEX.MX,2528.083736,3539.770679
Total,2803.959320,4433.296404


# Divisas

1500 dólares (peso dolar), 700 euros, y -600 libras esterlinas

Primero obtenemos el valor de cada Divisa en la fecha de Valoración

In [18]:
fecha_valoracion = '2023-03-10'
acciones = ['USDMXN=X', 'EURMXN=X', 'GBPMXN=X']
posiciones = [1500, 700, -600]
alpha = 0.98

stock_data = yf.download(acciones, end = fecha_valoracion)
X=stock_data.Close
X = X.dropna()
X = X[::-1]
X0 = X.head(1)
X0
DeltaX = (X.shift(1)/X)-1
DeltaX.iloc[0,:] = 0
X_s = X0.values*(1+DeltaX)
posiciones = pd.Series(posiciones,index = acciones)
PL = (X_s-X0.values[0])*posiciones
PL['Total']=PL.sum(axis=1)
vector_VaR = [PL.iloc[:,i].quantile(1-alpha) for i in range(len(acciones))]
vector_VaR
VaR = pd.Series(vector_VaR,index = acciones)
VaR = PL.quantile(alpha,numeric_only = True)
VaR

[*********************100%***********************]  3 of 3 completed


EURMXN=X    287.005379
GBPMXN=X    209.447395
USDMXN=X    470.193913
Total       508.435615
Name: 0.98, dtype: float64

In [19]:
#Multiplicamos por el número de posiciones para obtener el valor total
X0*posiciones

,EURMXN=X,GBPMXN=X,USDMXN=X
Date,,,
2023-03-09,13269.620132,-12776.580048,26956.126213


In [20]:
# Utilizamos la clase
Divisas=Riesgo_Equity_FX()

## Sin alisado

In [21]:
 Divisas.compute(['USDMXN=X', 'EURMXN=X', 'GBPMXN=X'], [1500, 700, -600], '2023-03-10', alisado=False) 

[*********************100%***********************]  3 of 3 completed


In [22]:
Divisas.summary

,VaR,CVaR
EURMXN=X,287.005379,417.217270
GBPMXN=X,209.447395,278.102401
USDMXN=X,470.193913,722.202763
Total,508.435615,765.503432


## Con Alisado 

In [23]:
Divisas.compute(['USDMXN=X', 'EURMXN=X', 'GBPMXN=X'], [1500, 700, -600], '2023-03-10', alisado=True) 


[*********************100%***********************]  3 of 3 completed


In [24]:
Divisas.summary

,VaR,CVaR
EURMXN=X,129.969972,231.919690
GBPMXN=X,162.190199,230.232397
USDMXN=X,336.287053,523.525814
Total,345.916134,536.414324


# Bonos

2 bonos largos, uno cupón cero de cetes (con la curva guber) con nocional de 1500 vencimiento de 180 días, un bono M con tasa fija de 6.5% con un cupon de 6.05% anual vencimiento de 3600 días y nocional de 1000, y un bono corto 1000 bondes con plazo 707 con cupón que paga 28 días (calcular cupón basado en curva de fondeo y descontar con la curva correspondiente).

## CETES

In [25]:
def Cete(tasa, t): # días del bono t = (fecha_entrega - fecha_valoración).days
  return 10/(1 + tasa*(t/360))

In [26]:
data_guber = "Tarea1-AdRF/tasa_guber_st.txt"
df= pd.read_table(data_guber)
df.head()
tasa_cete=df.iloc[0][5]
tasa_cete



0.02546244

In [27]:
# Muestra del valor histórico de las tasas del CETE
df.head()

,DATE,1,7,30,90,180,270,360,720,1080,...,1800,2160,2520,2880,3240,3600,5400,7200,9000,10800
0,20230228,0.024102,0.024112,0.024151,0.024752,0.025462,0.026221,0.026904,0.029372,0.031364,...,0.033511,0.034364,0.035266,0.036094,0.036941,0.037818,0.042174,0.045842,0.050106,0.054879
1,20230227,0.023542,0.024112,0.024151,0.024695,0.025521,0.026221,0.026965,0.029299,0.031218,...,0.033510,0.033908,0.034854,0.035929,0.037087,0.038289,0.041826,0.046116,0.048545,0.055219
2,20230224,0.023542,0.024112,0.024262,0.024752,0.025346,0.026161,0.026965,0.029298,0.031449,...,0.033871,0.034074,0.034800,0.036177,0.037613,0.038624,0.041508,0.045154,0.046868,0.052302
3,20230223,0.023820,0.024112,0.024262,0.024752,0.025346,0.026161,0.026965,0.029036,0.030921,...,0.032900,0.033412,0.034000,0.035356,0.036746,0.037398,0.040464,0.042949,0.046149,0.049407
4,20230222,0.023542,0.024112,0.024262,0.024922,0.025637,0.026573,0.027273,0.029469,0.031715,...,0.034136,0.034559,0.035129,0.036454,0.037809,0.038484,0.041525,0.043672,0.044949,0.050585


In [28]:
df1=pd.read_table(data_guber)
df1.head()

,DATE,1,7,30,90,180,270,360,720,1080,...,1800,2160,2520,2880,3240,3600,5400,7200,9000,10800
0,20230228,0.024102,0.024112,0.024151,0.024752,0.025462,0.026221,0.026904,0.029372,0.031364,...,0.033511,0.034364,0.035266,0.036094,0.036941,0.037818,0.042174,0.045842,0.050106,0.054879
1,20230227,0.023542,0.024112,0.024151,0.024695,0.025521,0.026221,0.026965,0.029299,0.031218,...,0.033510,0.033908,0.034854,0.035929,0.037087,0.038289,0.041826,0.046116,0.048545,0.055219
2,20230224,0.023542,0.024112,0.024262,0.024752,0.025346,0.026161,0.026965,0.029298,0.031449,...,0.033871,0.034074,0.034800,0.036177,0.037613,0.038624,0.041508,0.045154,0.046868,0.052302
3,20230223,0.023820,0.024112,0.024262,0.024752,0.025346,0.026161,0.026965,0.029036,0.030921,...,0.032900,0.033412,0.034000,0.035356,0.036746,0.037398,0.040464,0.042949,0.046149,0.049407
4,20230222,0.023542,0.024112,0.024262,0.024922,0.025637,0.026573,0.027273,0.029469,0.031715,...,0.034136,0.034559,0.035129,0.036454,0.037809,0.038484,0.041525,0.043672,0.044949,0.050585


In [29]:
# Posición del Bono Cupón cero (cete)
# Utilizamos la funcón Cete, donde ingresamos los valores de la tasa gubernamental
# de 180 días, y la temporalidad t=180
cete=Cete(tasa_cete,180)
ValuacionCete=cete*1500
ValuacionCete

14811.432395655778

In [30]:
# Creación de una clase para la valuación de cetes
class Riesgo_Cete:

  def __init__(self):
    self.posicion = None
    self.fecha_valoracion = None
    self.fecha_entrega = None
    self.plazo = None
    self.VaR = None
    self.CVaR = None
    self.alpha = None
    # self.summary = None
    
  
  def compute(self,curva_tasa, dias, posicion, alpha):
    """
    - curva_tasa: DataFrame
    """
    # CETE
    self.plazo = dias # REVISAR
    ult_tasa = curva_tasa[str(self.plazo)][0]
    df = pd.DataFrame({'Tasa':curva_tasa[str(self.plazo)]})
    df['Ret_Tasa'] = (df['Tasa'].shift(1)/df['Tasa']) - 1
    df['Ret_Tasa'][0] = 0
    print(ult_tasa)
    print(df)
    df['Tasa_Act'] = ult_tasa*(1+df['Ret_Tasa'])
    df['CETE'] = posicion*df['Tasa_Act'].apply(Cete, args = dias)
    ult_cete = df['CETE'][0]
    df['CETE_PL'] = ult_cete - df['CETE']
    self.VaR = df['CETE_PL'].quantile(alpha, numeric_only=False)
    self.CVaR = CVaR(df['CETE_PL'],self.VaR)
    self.alpha = alpha
    self.posicion = posicion
    self.fecha_valoracion = fval
    self.fecha_entrega = fent

In [31]:
miCETE=Riesgo_Cete()

In [32]:
# Ahora valuamos el Bono M, con la tasa Yield
tasayield="Tarea1-AdRF/tasa_yield.txt"
dfy= pd.read_table(tasayield)
dfy

,DATE,1,7,30,90,180,270,360,720,1080,...,1800,2160,2520,2880,3240,3600,5400,7200,9000,10800
0,20230228,0.080652,0.079589,0.078358,0.079767,0.079900,0.079655,0.079100,0.076726,0.077014,...,0.076732,0.076239,0.075860,0.075838,0.075919,0.075959,0.076348,0.076733,0.076970,0.077093
1,20230227,0.078777,0.079589,0.078358,0.079584,0.080083,0.079655,0.079281,0.076535,0.076657,...,0.076731,0.075227,0.074973,0.075492,0.076218,0.076905,0.075717,0.077193,0.074572,0.077571
2,20230224,0.078777,0.079589,0.078719,0.079767,0.079535,0.079472,0.079281,0.076531,0.077223,...,0.077556,0.075596,0.074858,0.076014,0.077300,0.077578,0.075143,0.075582,0.071996,0.073473
3,20230223,0.079707,0.079589,0.078719,0.079767,0.079535,0.079472,0.079281,0.075847,0.075927,...,0.075333,0.074126,0.073137,0.074288,0.075517,0.075117,0.073251,0.071891,0.070891,0.069406
4,20230222,0.078777,0.079589,0.078719,0.080316,0.080447,0.080726,0.080185,0.076978,0.077877,...,0.078163,0.076671,0.075565,0.076595,0.077702,0.077297,0.075173,0.073100,0.069049,0.071061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,20220303,0.080652,0.077256,0.074575,0.080133,0.083001,0.083886,0.083600,0.082954,0.086108,...,0.085702,0.082348,0.078857,0.076232,0.073988,0.072604,0.066394,0.057684,0.048696,0.060352
251,20220302,0.080407,0.078534,0.074575,0.079584,0.083008,0.083867,0.083568,0.083311,0.086220,...,0.086204,0.083230,0.079610,0.076566,0.074233,0.072844,0.066663,0.057765,0.048785,0.060507
252,20220301,0.080652,0.078849,0.074575,0.080499,0.083012,0.083861,0.083601,0.082981,0.085951,...,0.086108,0.082738,0.079058,0.076223,0.073927,0.072581,0.066268,0.057161,0.048488,0.061388
253,20220228,0.080652,0.077724,0.072594,0.077755,0.082819,0.084047,0.083961,0.082554,0.086184,...,0.085863,0.082703,0.079242,0.076097,0.073712,0.072370,0.065666,0.056420,0.048582,0.060277


In [33]:
# Escenario a 180 días
dias = 180
curva_tasa=dfy
posicion = 1500


plazo = dias 
ult_tasa = curva_tasa[str(plazo)][0]
df = pd.DataFrame({'Tasa':curva_tasa[str(plazo)]})
df['Ret_Tasa'] = (df['Tasa'].shift(1)/df['Tasa']) - 1
df['Ret_Tasa'][0] = 0
print(ult_tasa)
print(df)
df['Tasa_Act'] = ult_tasa*(1+df['Ret_Tasa'])

0.07990014
         Tasa  Ret_Tasa
0    0.079900  0.000000
1    0.080083 -0.002278
2    0.079535  0.006881
3    0.079535  0.000000
4    0.080447 -0.011338
..        ...       ...
250  0.083001 -0.019780
251  0.083008 -0.000078
252  0.083012 -0.000053
253  0.082819  0.002334
254  0.082454  0.004425

[255 rows x 2 columns]


## Bono M

In [34]:
# Definición de una función para la valuación del Bono M
def bonoMyield(x, plazos, plazocupon, tfcupon, nominal, contratos): #valoración bono tasa fija
  
 N=int(plazos/plazocupon)+1   
 a=(1-(1+x*plazocupon/360)**(-N))/(plazocupon*x/360)
 p1=plazos-plazocupon*(N-1)   
 Vty=((contratos*nominal*tfcupon*plazocupon/360)*a+(contratos*nominal)/((1+x*plazocupon/360)**N))*(1+x*plazocupon/360)**(1-p1/plazocupon) 

 return Vty 

In [35]:
# Bono M con tasa fija de 6.5% con un cupón anual de 6.05% anual vencimiento de 3600 días
# y nocional de 1000
x = bonoMyield(.0605,3600,182,0.065,100,1000)
x

104053.06525595774

Hasta aquí va bien

In [ ]:
def BonoM(x, plazos, plazocupon, tfcupon, nominal, contratos):
  x=x0  
  N=int(plazos/plazocupon)+1   
  a=(1-(1+x*plazocupon/360)**(-N))/(plazocupon*x/360)
  p1=plazos-plazocupon*(N-1)   
  precio=((contratos*nominal*tfcupon*plazocupon/360)*a+(contratos*nominal)/((1+x*plazocupon/360)**N))*(1+x*plazocupon/360)**(1-p1/plazocupon) 

  return precio

In [ ]:
# Clase Auxiliar

class Riesgo_Fixed_Income:

  def __init__(self):
    self.posiciones = None
    self.VaR = None
    self.CVaR = None
    self.alpha = None
    self.summary = None
    self.ordenado = None
    self.alisado = None
    
  def compute(self, cete, bonom, bonde, alpha):
    """
    Donde cete, bonom, bonde son diccionarios con las siguientes características:
    cete = {'Posicion':posicion_c, 'Plazo':plazo, 'Tasa':curva_tasa}
    bonom = {'Posicion:'posicion_bm, 'Tasa_fija':fija,'Tasa_flotante':flotante, 'Plazo_fijo':plazo_fijo,\
    'Plazo_acum':plazo_acum}
    bonde = {'Posicion':posicion_bd, 'Tasas':tasas_df}
    """
    posiciones = [cete['Posicion'],bonom['Posicion'],bonde['Posicion']]
    # Pegar las tasas con las fechas adecuadas
    X = cete['Tasa'][['Fecha',str(cete['Plazo'])]].merge(bonom['Tasa'],how = 'inner',on= 'Fecha') # Combinar tasas
    X = X.merge(bonde['Tasas'],how = 'inner',on = 'Fecha')

    X0 = X.head(1)
    DeltaX = np.log(X.shift(-1, fill_value=1)/X)
    X_s=np.exp(DeltaX.squeeze())*X0.squeeze()
    # Y_s = ['Cete', 'BonoM','BondesD']
    # Y_s = aplicar valoracion de cete a lo de cete, bonom a lo de bonom y bonde a lo de bonde
    Y0 = Y_s.head(1)
    PL = Y0 - Y_s
    cols=PL.shape[1]
    PL['Total']=PL.sum(axis=1)
    VaR=PL.quantile(alpha, numeric_only=False)
    VaR_C=[]

    for i in range(0,PL.shape[1]):
      VC=CVaR(PL.iloc[:,i],VaR[i])
      VaR_C.append(VC)

    self.summary = pd.DataFrame({"VaR":VaR})
    self.summary['CVaR'] = VaR_C

    self.posiciones = {'Cete':cete['Posicion'],'BonoM':bonom['Posicion'],'BondeD':bonde['Posicion']}
    self.VaR = self.summary[['VaR']]
    self.CVaR = self.summary[['CVaR']]
    self.alpha = alpha
    return



################################################################
  def compute(self, curva_tasa, fval, fent, posicion, alpha):
    """
    - curva_tasa: DataFrame
    """
    # CETE
    self.plazo = (fent - fval).days 
    ult_tasa = curva_tasa[str(self.plazo)][0]
    df = pd.DataFrame({'Tasa':curva_tasa[str(self.plazo)]})
    df['Ret_Tasa'] = (df['Tasa'].shift(-1, fill_value=0)/df['Tasa']) - 1
    df['Tasa_Act'] = ult_tasa*(1+df['Ret_Tasa'])
    df['CETE'] = posicion*df['Tasa_Act'].apply(Cete, args = (self.plazo))
    ult_cete = df['CETE'][0]
    df['CETE_PL'] = ult_cete - df['CETE']
    self.VaR = df['CETE_PL'].quantile(alpha, numeric_only=False)
    self.CVaR = CVaR(df['CETE_PL'],self.VaR)
    self.alpha = alpha
    self.posicion = posicion
    self.fecha_valoracion = fval
    self.fecha_entrega = fent
   

    return

In [ ]:
class Riesgo_Cete:

  def __init__(self):
    self.posicion = None
    self.fecha_valoracion = None
    self.fecha_entrega = None
    self.plazo = None
    self.VaR = None
    self.CVaR = None
    self.alpha = None
    # self.summary = None
    
  
  def compute(self, curva_tasa, dias, posicion, alpha):
    """
    - curva_tasa: DataFrame
    """
    # CETE
    self.plazo = dias # REVISAR
    ult_tasa = curva_tasa[str(self.plazo)][0]
    df = pd.DataFrame({'Tasa':curva_tasa[str(self.plazo)]})
    df['Ret_Tasa'] = (df['Tasa'].shift(1)/df['Tasa']) - 1
    df['Ret_Tasa'][0] = 0
    print(ult_tasa)
    print(df)
    df['Tasa_Act'] = ult_tasa*(1+df['Ret_Tasa'])
    df['CETE'] = posicion*df['Tasa_Act'].apply(Cete, args = dias)
    ult_cete = df['CETE'][0]
    df['CETE_PL'] = ult_cete - df['CETE']
    self.VaR = df['CETE_PL'].quantile(alpha, numeric_only=False)
    self.CVaR = CVaR(df['CETE_PL'],self.VaR)
    self.alpha = alpha
    self.posicion = posicion
    self.fecha_valoracion = fval
    self.fecha_entrega = fent
 

    return

In [ ]:
dias = 180
curva_tasa=df
posicion = 1500


plazo = dias 
ult_tasa = curva_tasa[str(plazo)][0]
df = pd.DataFrame({'Tasa':curva_tasa[str(plazo)]})
df['Ret_Tasa'] = (df['Tasa'].shift(1)/df['Tasa']) - 1
df['Ret_Tasa'][0] = 0
print(ult_tasa)
print(df)
df['Tasa_Act'] = ult_tasa*(1+df['Ret_Tasa'])

0.015620437
         Tasa  Ret_Tasa
0    0.015620  0.000000
1    0.015656 -0.002278
2    0.015549  0.006881
3    0.015549  0.000000
4    0.015727 -0.011338
..        ...       ...
250  0.016227 -0.019780
251  0.016228 -0.000078
252  0.016229 -0.000053
253  0.016191  0.002334
254  0.016120  0.004425

[255 rows x 2 columns]


In [ ]:
cet = [Cete(i,) for i in df['Tasa_Act']]
df['CETE'] = posicion*df['Tasa_Act'].apply(Cete, args = dias)

TypeError: ignored

In [ ]:
miCETE = Riesgo_Cete()

In [ ]:
# Función para valuar el Bono M
def BonoM(x, plazos, plazocupon, tfcupon, nominal, contratos):
  x=x0  
  N=int(plazos/plazocupon)+1   
  a=(1-(1+x*plazocupon/360)**(-N))/(plazocupon*x/360)
  p1=plazos-plazocupon*(N-1)   
  precio=((contratos*nominal*tfcupon*plazocupon/360)*a+(contratos*nominal)/((1+x*plazocupon/360)**N))*(1+x*plazocupon/360)**(1-p1/plazocupon) 

  return precio

En R

In [36]:
# Primero vamos a definir una función para la interpolación
%%R
#FUNCIÓN DE INTERPOLACIÓN ALAMBRADA

talamb=function(nodos,curva,plazos){
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n){
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}

# Función para diagonalizar un vector
diagv=function(x)	{
  n01=nrow(as.matrix(x))
  m01=ncol(as.matrix(x))
  dimmax=max(n01,m01)
  res=matrix(0,dimmax,dimmax)
  for (i in 1:dimmax){
    res[i,i]=x[i]
  }
  res
}

#Función de cuantil más cercano
equantile <- function(v,p=.5,ns=nrow(as.matrix(v))) 
{ 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  ranking <- order(v) 
  vw=matrix(0,ns,1)
  vw[1:ns]=seq(1/ns,ns)  
  sumw <- cumsum(vw[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
} 

In [37]:
%%R
#VaR con alisado

wquantile <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
} 

#CVaR con alisado
wcvar <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  loss= v [ ranking [ which( plist <= p ) ] ]  
  esc=w [ ranking [ which( plist <= p ) ] ]  
  sum(loss*esc)/(sum(esc))
}

In [38]:
%%R
#Definición de parámetros para valorar
fval=as.Date("20230310",format="%Y%m%d")
alpha = 0.98 #@param {type:"slider", min:0, max:1, step:0.01}
#poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada
itpl = 0 #@param ["0", "1"] {type:"raw"} 

Valuación del Bono M en R

In [39]:
# Bono M
%%R
tasayield="Tarea1-AdRF/tasa_yield.txt"


itpl=0 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada

tf = cbind(0.065,0)
tfcupon = cbind(0.0605, 0)
plazos= cbind(3600, 0)
plazocupon = cbind(182, 0)
contratos= cbind(1, 0)
nominal = 1000

In [40]:
%%R

#carga de datos
#carga de tasas de descuento
data1<-read.table(tasayield)
n<-nrow(data1)
m_orig=ncol(data1)
X_orig=data.frame(data1[2:n,2:m_orig])
nodos=data.frame(data1[1,2:m_orig])
n=n-1
#Posición inicial
#interpolación de tasas y volatilidades
m=ncol(plazos)
X=matrix(0,n-1,m)
for (i in 1:(n-1))
{
X[i,]=if(itpl==0){approx(nodos,X_orig[i,],plazos)$y}else{talamb(nodos,X_orig[i,],plazos)}
}

In [41]:
%%R

#Función de valoración por tasa yield
bonoMyield=function(x, plazos, plazocupon, tfcupon, nominal, contratos) #valoración bono tasa fija
{
x=x0  
N=as.integer(plazos/plazocupon)+1   
a=(1-(1+x*plazocupon/360)^(-N))/(plazocupon*x/360)
p1=plazos-plazocupon*(N-1)   
((contratos*nominal*tfcupon*plazocupon/360)*a+(contratos*nominal)/((1+x*plazocupon/360)^N))*(1+x*plazocupon/360)^(1-p1/plazocupon)   
}

x0=0.065 #tasas de descuento valor actual
V0_bonom = bonoMyield(x0,plazos, plazocupon, tfcupon, nominal, contratos)
V0_bonom[1,2] = 0
VT0_bonom=sum(V0_bonom)
print("Valor del Bono M")
VT0_bonom

[1] "Valor del Bono M"
[1] 973.9283


Valuación de CETE en R

In [42]:
%%R
library(dplyr)
base=("Tarea1-AdRF/tasa_guber.txt")
data_cetes<-read.table(base)
colnames(data_cetes)=data_cetes[1,]
data_cetes= data_cetes[-1,]
colnames(data_cetes)
data_cetes=data_cetes[,c('DATE', '180')]
data_cetes['Rendimientos']= (data_cetes[ "180"]/lag(data_cetes[ "180"], 1) )-1
data_cetes[is.na(data_cetes)]=0
data_cetes['Estimación escenario']= data_cetes[1,'180']*(1+data_cetes['Rendimientos'])
posicion=1500
plazo=180
data_cetes['Valor de la posicion']= (posicion*10)/(1+data_cetes['Estimación escenario']*plazo/360)
data_cetes[is.na(data_cetes)]=0
data_cetes['PL']=data_cetes['Valor de la posicion']-data_cetes[1,'Valor de la posicion']
print('Valor de la posición')
print(data_cetes[1,'Valor de la posicion'])

Attaching package: ‘dplyr’



    filter, lag



    intersect, setdiff, setequal, union




[1] "Valor de la posición"
[1] 14423.89


Valuación de BONDES en R

In [43]:
%%R
#install.packages("dplyr")
require(dplyr)
require(data.table)




Attaching package: ‘data.table’



    between, first, last




In [45]:
%%R
# Cargamos las tasas correspondientes
btasadescst="Tarea1-AdRF/tasa_guber_st.txt"
btasafondeo="Tarea1-AdRF/tfondeo.txt"
plazos= cbind(707, 707) #Vencimiento del bono
plazocupon= cbind(28, 28) #plazos_bdm fijos de cada cupón
contratos = cbind(-1000, 0) # 1000 Bondes
nominal=100

In [46]:

%%R
#carga de datos
#carga de tasas de descuento

data1<-read.table(btasadescst)
n<-nrow(data1)
m_orig=ncol(data1)
X_orig=data.frame(data1[2:n,2:m_orig])
X1_orig=mutate(data.frame(data1[2:n,1:m_orig]), V1 = as.Date(V1,format="%Y%m%d"))
nodos=data.frame(data1[1,2:m_orig])
n=n-1

data3<-read.table(btasadescst)
n3<-nrow(data3)
m3_orig=ncol(data3)
X3_orig=data.frame(data3[2:n3,2:m3_orig])
X3a_orig=mutate(data.frame(data1[2:n,1:m_orig]),V1=as.Date(V1,format="%Y%m%d"))
nodos3=data.frame(data3[1,2:m3_orig])
n3=n3-1

data2<-read.table(btasafondeo)
n2<-nrow(data2)
X2_orig=data.frame(data2[2:n2,1:2])
X2a_orig=mutate(X2_orig, V1=as.Date(V1,format="%Y%m%d"), V2=as.numeric(as.character(V2)))
tfh=seq(as.Date("20230310",format="%Y%m%d"), as.Date("20230310",format="%Y%m%d"), "days") #sucesión de dias para tasa fondeo
#tfhd=data.frame(ID=1:count(tfh),fecha=tfh)
tfhd=CJ(fecha=tfh)

In [47]:
%%R
#Cruzar la sucesión de todos los días versus el de tasa de fondeo
tfhd=setDT(tfhd)[, join_date := tfh][order(-join_date)]
X2a_orig=setDT(X2a_orig)[, join_date := V1][order(-join_date)]
X2a_orig=X2a_orig[tfhd, on = .(join_date), roll = Inf]

In [48]:
%%R
#Cruzar la sucesión de todos los días versus el de tasa de fondeo
#buscar fecha de valuación en tfondeo
tf_act=X2a_orig[fecha==fval,]$V2/100
tf_int=X2a_orig[fecha<=fval & fecha>=(fval-plazocupon[1])]$V2/100

X1_orig=setDT(X1_orig)[, join_date := V1][order(-join_date)] #Para alinear con valor presente y tasa de fondeo.
X2_pr=X2a_orig[X1_orig, on = .(join_date)] #Se alinea la tasa de fondeo por fecha con la tasa de valor presente.
X2_pr

       V1 V2  join_date fecha       i.V1       i.V2         V3         V4
  1: <NA> NA 2023-02-28  <NA> 2023-02-28 0.02410237 0.02411239 0.02415083
  2: <NA> NA 2023-02-27  <NA> 2023-02-27 0.02354185 0.02411239 0.02415083
  3: <NA> NA 2023-02-24  <NA> 2023-02-24 0.02354185 0.02411239 0.02426187
  4: <NA> NA 2023-02-23  <NA> 2023-02-23 0.02381987 0.02411239 0.02426187
  5: <NA> NA 2023-02-22  <NA> 2023-02-22 0.02354185 0.02411239 0.02426187
 ---                                                                     
251: <NA> NA 2022-03-03  <NA> 2022-03-03 0.02410237 0.02340548 0.02298493
252: <NA> NA 2022-03-02  <NA> 2022-03-02 0.02402909 0.02379274 0.02298493
253: <NA> NA 2022-03-01  <NA> 2022-03-01 0.02410237 0.02388809 0.02298493
254: <NA> NA 2022-02-28  <NA> 2022-02-28 0.02410237 0.02354739 0.02237422
255: <NA> NA 2022-02-25  <NA> 2022-02-25 0.02410237 0.02368733 0.02237422
             V5         V6         V7         V8         V9        V10
  1: 0.02475177 0.02546244 0.02622075 0.0

In [49]:
%%R
#Verificamos el plazo del cupón
plazocupon

     [,1] [,2]
[1,]   28   28


In [50]:
%%R
# Matriz de ceros
matrix(0,1,sum(26) )

     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    0    0    0    0    0    0    0    0    0     0     0     0     0     0
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     0     0     0     0     0     0     0     0     0     0     0     0


In [ ]:
%%R      

m=ncol(plazos)


N=as.integer(plazos/plazocupon)+1 #número de cupones a pagar
VTplazos=matrix(0,1,sum(N)) #vector de todos los plazos de todos los contratos
contratosT=matrix(0,1,sum(N)) #vector de todos los contratos de todos los flujos de todos los contratos
nominalT=matrix(0,1,sum(N)) #vector de todos los nominales de todos los flujos de todos los contratos
plazocuponT=matrix(0,1,sum(N)) #vector de todos los plazoscupon de todos los flujos de todos los contratos
tasafijaT=matrix(0,1,sum(N)) #vector de tasas fijas de todos los flujos de todos los contratos
ulNomT=matrix(0,1,sum(N)) #vector de contratos a final de flujo


plazini=plazos-plazocupon*(N-1) #vector de plazos iniciales
ddv=plazocupon-plazini #dias trasncurridos del cupón vigente
tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
tfcupondev=matrix(0,1,m) #cupón de los días devengados
tfcupgen=((1+tf_act/360)^(plazocupon[1])-1)*360/plazocupon[1] #el segundo al último cupón de todos los bonos
#calcula cupones de bonos
for (j in (1:m))
{
  tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
  tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon[1]-ddv[j])-1)*360/plazocupon[1]
}


for (j in (1:m))
{
  if (j==1)
  {
    VTplazos[,1:sum(N[1:j])]=seq(plazini[j],plazos[j], by=plazocupon[j])
    contratosT[,1:sum(N[1:j])]=seq(contratos[j],contratos[j])
    plazocuponT[,1:sum(N[1:j])]=seq(plazocupon[j],plazocupon[j])
    ulNomT[,sum(N[1:j])]=contratos[j]
    tasafijaT[,1]=tfcupon[j]
    tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
  }
  else
  {
    VTplazos[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazini[j],plazos[j], by=plazocupon[j])
    contratosT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(contratos[j],contratos[j])					
    plazocuponT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazocupon[j],plazocupon[j])
    tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
    tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    ulNomT[,sum(N[1:j])]=contratos[j]
  }
}

Xvp=matrix(0,n,ncol(VTplazos))
Xst=matrix(0,n,ncol(VTplazos))

for (i in (1:n))
{
  Xvp[i,]=if(itpl==0){approx(nodos,X_orig[i,],VTplazos)$y}else{talamb(nodos,X_orig[i,],VTplazos)}
  Xst[i,]=if(itpl==0){approx(nodos3,X3_orig[i,],VTplazos)$y}else{talamb(nodos3,X3_orig[i,],VTplazos)}
  
}

head(Xvp)
head(Xst)


In [58]:
%%R
tf_act

[1] 0.1105


In [52]:
%%R   

X_bd_tc=matrix(1,n,ncol(contratosT))*X2_pr$V2/100
X_bd_ext=cbind(X_bd_tc,as.matrix(Xvp),as.matrix(Xst))


bondeD=function(contratosT, nominal, tasafijaT, plazocuponT, VTplazos, Xvp, Xst, N)
{ 
  V0=matrix(0,1,nrow(as.matrix(N)))
  V0f=((((contratosT*(tasafijaT)*(plazocuponT/360))+ulNomT)/(1+(Xvp+Xst)*VTplazos/360)))*nominal
  for (j in (1:nrow(as.matrix(N))))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
} 

V0_bd=bondeD(contratosT, nominal, tasafijaT, plazocuponT, VTplazos, Xvp[1,], Xst[1,], N)
print("Valor del Bonde D")
print(V0_bd[1,1])

[1] "Valor del Bonde D"
[1] NA


## VaR y CVaR Sin Alisado


In [59]:
%%R
#DIMENSION DE TODOS LOS INSTRUMENTOS
#Son 3 instrumentos financieros 
n_if=matrix(0,2,1)
n_if[1]=ncol(X_bd_ext) #bonde
n_if[1]=1 #Cetes
n_if[2]=0 #Bonos M
#valor del portafolios

V0_port=cbind( VT0_bonom)

V0T_port=sum(V0_port)

In [60]:
%%R
#INTEGRACIÓN DE TODOS LOS FACTORES DE RIESGO EN UNA MATRIZ
X_port=cbind(x[,1], X[,1]) #Factores de riesgo del portafolios de 8(9) instrumentos financieros

#Cálculo de variaciones Delta_X DEL PORTAFOLIOS
DeltaX_port=as.matrix(log(X_port[1:(n-1)]/X_port[2:(n)]))
DeltaX_port[is.nan(DeltaX_port)] <- 0 #quitamos NaN
DeltaX_port[is.na(DeltaX_port)] <- 0 #quitamos Na
DeltaX_port[is.infinite(DeltaX_port)] <- 0 #quitamos Na


Ns=nrow(DeltaX_port) #Definimos número de escenarios históricos

DeltaX_s=DeltaX_port


Error in cbind(x[, 1], X[, 1]) : object 'x' not found


RInterpreterError: ignored

In [61]:
%%R
#Función de cuantil más cercano
equantile <- function(v,p=.5,ns=nrow(as.matrix(v))) 
{ 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  ranking <- order(v) 
  vw=matrix(0,ns,1)
  vw[1:ns]=seq(1/ns,ns)  
  sumw <- cumsum(vw[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
} 

### CETES

In [62]:
%%R
print('Valor de la posición')
print(data_cetes[1,'Valor de la posicion'])
VaR= quantile(data_cetes$PL,0.98 ) 
print('VaR')
print(VaR)
CVaR=mean(data_cetes$PL[data_cetes['PL']>VaR])
print('CVaR')
print(CVaR)

[1] "Valor de la posición"
[1] 14423.89
[1] "VaR"
     98% 
8.229372 
[1] "CVaR"
[1] 12.13667


### Bono M

En este caso tenemos que el Bono M no presenta riesgo, puesto que desde el inicio ya sabemos como va evolucionando en el tiempo.

## VaR y CVaR con Alisado

### CETES

In [63]:
%%R
wquantile <- function(v,w=rep(1,length(v)),p=.5){ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
}
#CVaR con alisado
wcvar <- function(v,w=rep(1,length(v)),p=.5) { 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  loss= v [ ranking [ which( plist < p ) ] ]  
  esc=w [ ranking [ which( plist < p ) ] ]  
  sum(loss*esc)/(sum(esc))
}

VaR_alisado=wquantile(data_cetes$PL)
print('VaR_alisado')
print(VaR_alisado)
CVaR_alisado=wcvar(data_cetes$PL)
print('CVaR_alisado')
print(CVaR_alisado)


[1] "VaR_alisado"
[1] 0
[1] "CVaR_alisado"
[1] -2.073684


### Bono M

En este caso tenemos que el Bono M no presenta riesgo, puesto que desde el inicio ya sabemos como va evolucionando en el tiempo.

###Bondes

In [64]:
%%R
#install.packages("dplyr")
require(dplyr)
require(data.table)

In [65]:
%%R
btasadescst="Tarea1-AdRF/tasa_guber_st.txt"
btasafondeo="Tarea1-AdRF/tfondeo.txt"
plazos= cbind(707, 707) #Vencimiento del bono
plazocupon= cbind(28, 28) #plazos_bdm fijos de cada cupón
contratos = cbind(-1000, 0)
nominal=100

In [66]:
%%R
#carga de datos
#carga de tasas de descuento
data1<-read.table(btasadescst)
n<-nrow(data1)
m_orig=ncol(data1)
X_orig=data.frame(data1[2:n,2:m_orig])
X1_orig=mutate(data.frame(data1[2:n,1:m_orig]), V1=as.Date(V1,format="%Y%m%d"))
nodos=data.frame(data1[1,2:m_orig])
n=n-1

data3<-read.table(btasadescst)
n3<-nrow(data3)
m3_orig=ncol(data3)
X3_orig=data.frame(data3[2:n3,2:m3_orig])
X3a_orig=mutate(data.frame(data1[2:n,1:m_orig]), V1=as.Date(V1,format="%Y%m%d"))
nodos3=data.frame(data3[1,2:m3_orig])
n3=n3-1

data2<-read.table(btasafondeo)
n2<-nrow(data2)
X2_orig=data.frame(data2[2:n2,1:2])
X2a_orig=mutate(X2_orig, V1=as.Date(V1,format="%Y%m%d"), V2=as.numeric(as.character(V2)))
tfh=seq(as.Date("20220831",format="%Y%m%d"), as.Date("20220831",format="%Y%m%d"), "days") #sucesión de dias para tasa fondeo
#tfhd=data.frame(ID=1:count(tfh),fecha=tfh)
tfhd=CJ(fecha=tfh)

In [67]:
%%R
#Cruzar la sucesión de todos los días versus el de tasa de fondeo
tfhd=setDT(tfhd)[, join_date := tfh][order(-join_date)]
X2a_orig=setDT(X2a_orig)[, join_date := V1][order(-join_date)]
X2a_orig=X2a_orig[tfhd, on = .(join_date), roll = Inf]

In [69]:
%%R
#Cruzar la sucesión de todos los días versus el de tasa de fondeo
#buscar fecha de valuación en tfondeo
tf_act=X2a_orig[fecha==fval,]$V2/100
tf_int=X2a_orig[fecha<=fval & fecha>=(fval-plazocupon[1])]$V2/100

X1_orig=setDT(X1_orig)[, join_date := V1][order(-join_date)] #Para alinear con valor presente y tasa de fondeo.
X2_pr=X2a_orig[X1_orig, on = .(join_date)] #Se alinea la tasa de fondeo por fecha con la tasa de valor presente.
X2_pr

       V1 V2  join_date fecha       i.V1       i.V2         V3         V4
  1: <NA> NA 2023-02-28  <NA> 2023-02-28 0.02410237 0.02411239 0.02415083
  2: <NA> NA 2023-02-27  <NA> 2023-02-27 0.02354185 0.02411239 0.02415083
  3: <NA> NA 2023-02-24  <NA> 2023-02-24 0.02354185 0.02411239 0.02426187
  4: <NA> NA 2023-02-23  <NA> 2023-02-23 0.02381987 0.02411239 0.02426187
  5: <NA> NA 2023-02-22  <NA> 2023-02-22 0.02354185 0.02411239 0.02426187
 ---                                                                     
251: <NA> NA 2022-03-03  <NA> 2022-03-03 0.02410237 0.02340548 0.02298493
252: <NA> NA 2022-03-02  <NA> 2022-03-02 0.02402909 0.02379274 0.02298493
253: <NA> NA 2022-03-01  <NA> 2022-03-01 0.02410237 0.02388809 0.02298493
254: <NA> NA 2022-02-28  <NA> 2022-02-28 0.02410237 0.02354739 0.02237422
255: <NA> NA 2022-02-25  <NA> 2022-02-25 0.02410237 0.02368733 0.02237422
             V5         V6         V7         V8         V9        V10
  1: 0.02475177 0.02546244 0.02622075 0.0

In [70]:
%%R      

m=ncol(plazos)


N=as.integer(plazos/plazocupon)+1 #número de cupones a pagar
VTplazos=matrix(0,1,sum(N)) #vector de todos los plazos de todos los contratos
contratosT=matrix(0,1,sum(N)) #vector de todos los contratos de todos los flujos de todos los contratos
nominalT=matrix(0,1,sum(N)) #vector de todos los nominales de todos los flujos de todos los contratos
plazocuponT=matrix(0,1,sum(N)) #vector de todos los plazoscupon de todos los flujos de todos los contratos
tasafijaT=matrix(0,1,sum(N)) #vector de tasas fijas de todos los flujos de todos los contratos
ulNomT=matrix(0,1,sum(N)) #vector de contratos a final de flujo


plazini=plazos-plazocupon*(N-1) #vector de plazos iniciales
ddv=plazocupon-plazini #dias trasncurridos del cupón vigente
tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
tfcupondev=matrix(0,1,m) #cupón de los días devengados
tfcupgen=((1+tf_act/360)^(plazocupon[1])-1)*360/plazocupon[1] #el segundo al último cupón de todos los bonos
#calcula cupones de bonos
for (j in (1:m))
{
  tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
  tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon[1]-ddv[j])-1)*360/plazocupon[1]
}


for (j in (1:m))
{
  if (j==1)
  {
    VTplazos[,1:sum(N[1:j])]=seq(plazini[j],plazos[j], by=plazocupon[j])
    contratosT[,1:sum(N[1:j])]=seq(contratos[j],contratos[j])
    plazocuponT[,1:sum(N[1:j])]=seq(plazocupon[j],plazocupon[j])
    ulNomT[,sum(N[1:j])]=contratos[j]
    tasafijaT[,1]=tfcupon[j]
    tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
  }
  else
  {
    VTplazos[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazini[j],plazos[j], by=plazocupon[j])
    contratosT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(contratos[j],contratos[j])					
    plazocuponT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazocupon[j],plazocupon[j])
    tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
    tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    ulNomT[,sum(N[1:j])]=contratos[j]
  }
}

Xvp=matrix(0,n,ncol(VTplazos))
Xst=matrix(0,n,ncol(VTplazos))

for (i in (1:n))
{
  Xvp[i,]=if(itpl==0){approx(nodos,X_orig[i,],VTplazos)$y}else{talamb(nodos,X_orig[i,],VTplazos)}
  Xst[i,]=if(itpl==0){approx(nodos3,X3_orig[i,],VTplazos)$y}else{talamb(nodos3,X3_orig[i,],VTplazos)}
  
}

head(Xvp)
head(Xst)


  replacement has length zero


ℹ In argument: `V2 = as.numeric(as.character(V2))`.
Caused by warning:
! NAs introduced by coercion 




Error in tfcupon[j] <- ((1 + tfcupondev[j] * ddv[j]/360) * (1 + tf_act/360)^(plazocupon[1] -  : 
  replacement has length zero


RInterpreterError: ignored

In [71]:
%%R   

X_bd_tc=matrix(1,n,ncol(contratosT))*X2_pr$V2/100
X_bd_ext=cbind(X_bd_tc,as.matrix(Xvp),as.matrix(Xst))


bondeD=function(contratosT, nominal, tasafijaT, plazocuponT, VTplazos, Xvp, Xst, N)
{ 
  V0=matrix(0,1,nrow(as.matrix(N)))
  V0f=((((contratosT*(tasafijaT)*(plazocuponT/360))+ulNomT)/(1+(Xvp+Xst)*VTplazos/360)))*nominal
  V0f[1]=0
  V0f[27]=0
  for (j in (1:nrow(as.matrix(N))))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
} 

V0_bd=bondeD(contratosT, nominal, tasafijaT, plazocuponT, VTplazos, Xvp[1,], Xst[1,], N)
print("Valor del Bonde D")
print(V0_bd[1,1])

[1] "Valor del Bonde D"
[1] 0


# Futuros

100 contratos de compra de futuros de peso dólar con un strike de 20.83 vencimiento de 5 días, y 50 contratos de venta de futuros del IPC con strike de 49525 vencimiento de 53 días.

In [106]:
%reload_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/JuanPabloAS11/Tarea1-AdRF"

fatal: destination path 'Tarea1-AdRF' already exists and is not an empty directory.


In [4]:
# Definimos la función Forward
def Forward(tipo,M,S,K,vencimiento,tx,ty):
  if tipo == 'Cambio':
    precio = M*((S*((1+(tx*vencimiento/360))/(1+(ty*vencimiento/360)))- K)/(1+(tx*vencimiento/360)))
    return precio
  elif tipo == 'Indice':
    precio = M*(( S*(1+((tx-ty)*vencimiento/360))   - K)/(1+(tx*vencimiento/360)))
    return precio
  else:
    return 'Tipo de Forward incorrecto'
def intlin(x,X,Y):
    n = len(X)
    for i in range(n-1):
        if (X[i]<= x <= X[i+1]):
            a = Y[i+1] - Y[i]
            b = X[i+1] - X[i]
            return Y[i] + (x - X[i])*(a/b)
    return "No es posible interpolar"
class Riesgo_Forward:

  def __init__(self):
    self.posiciones = None
    self.strikes = None
    self.precios = None
    self.VaR = None
    self.CVaR = None
    self.summary = None
    self.precios = None
    self.alisado = None
    self.alpha = None

  def compute(self,M,S,K,vencimientos, tx,ty,alpha = 0.98, alisado = False):
    l = []
    for i in range(2):
      string = 'cambio' if i == 0 else 'indice'
      aux1 = pd.DataFrame()
      aux2 = pd.DataFrame()
      aux1['DATE'] = tx[i]['DATE']
      aux2['DATE'] = ty[i]['DATE']

      if (str(vencimientos[i]) in tx[i].columns):
        aux1['tx_'+string] = tx[i][str(vencimientos[i])]
      else:
        X = list(tx[i].columns)
        X.remove('DATE')
        X = [int(i) for i in X]
        aux1['tx_'+string] = [intlin(vencimientos[i],X, tx[i].iloc[j,1:].values) for j in range(tx[i].shape[0])]

      if (str(vencimientos[i]) in ty[i].columns):
        aux2['ty_'+string] = ty[i][str(vencimientos[i])]
      else:
        X = list(ty[i].columns)
        X.remove('DATE')
        X = [int(i) for i in X]
        aux2['ty_'+string] = [intlin(vencimientos[i],X, ty[i].iloc[j,1:].values) for j in range(ty[i].shape[0])]
      c = aux1.merge(aux2,on = 'DATE',how = 'inner')
      l.append(c)
    df = l[0].merge(l[1],on = 'DATE',how = 'inner')
    df = df.merge(S, how = 'inner',on = 'DATE')
    df = df.set_index('DATE')
    X = df.copy()
    X0 = X.head(1)
    DeltaX = (df.shift(1)/df)-1
    DeltaX.iloc[0,:] = 0
    X_s = X0.values*(1+DeltaX)

    Y_s = pd.DataFrame()
    Y_s['Cambio'] = [Forward('Cambio',M[0],X_s['USDMXN=X'][i],K[0],vencimientos[0],X_s['tx_cambio'][i],X_s['ty_cambio'][i])for i in range(X_s.shape[0])]
    Y_s['Indice'] = [Forward('Indice',M[0],X_s['^MXX'][i],K[1],vencimientos[1],X_s['tx_indice'][i],X_s['ty_indice'][i])for i in range(X_s.shape[0])]
    Y0 = Y_s.head(1)
    Y0 = Y0.values[0]
    PL = Y_s - Y0
    PL['Total']=PL.sum(axis=1)

    VaR_C=[]

    if alisado:
      c = 1
      PL.dropna(inplace = True)
      #PL = PL.iloc[::-1]
      PL["Esc_SA"] = ExponentialWeighted(0.05,PL.shape[0])
      col_aux = list(PL.columns)
      col_aux.remove("Esc_SA")
      vector_VaR = [VaR_CA(PL.iloc[:,i],PL.Esc_SA,alpha) for i in range(PL.shape[1]-1)]
      VaR = pd.Series(vector_VaR,index = col_aux)
    else:
      c = 0
      vector_VaR = [PL.iloc[:,i].quantile(alpha) for i in range(2)]
      VaR = pd.Series(vector_VaR,index = ['Cambio','Indice'])
      VaR = PL.quantile(alpha,numeric_only = True)
    
    for i in range(0,PL.shape[1]-c):
      VC=CVaR(PL.iloc[:,i],VaR[i])
      VaR_C.append(VC)

    self.summary = pd.DataFrame({"VaR":VaR})
    self.summary['CVaR'] = VaR_C

    self.posiciones = M
    self.strikes = K
    self.VaR = self.summary[['VaR']]
    self.CVaR = self.summary[['CVaR']]
    self.alpha = alpha
    self.alisado = alisado
    self.precios = list(Y0)

    return

miForward = Riesgo_Forward()

Libor=pd.read_table("Tarea1-AdRF/tasa_libor.txt")
Libor = Libor.rename(columns = {'DATE ':'DATE'})
Libor['DATE'] =Libor['DATE'].apply(lambda x: str(x))
Libor['DATE'] = Libor['DATE'].apply(lambda x: datetime(int(x[:4]), int(x[4:6]), int(x[6:])))
nombre = list(Libor.columns)
nombre.remove('DATE')
Libor[nombre] = Libor[nombre]/100

Guber= pd.read_table("Tarea1-AdRF/tasa_guber.txt")
Guber = Guber.rename(columns = {'DATE ':'DATE'})
Guber['DATE'] =Guber['DATE'].apply(lambda x: str(x))
Guber['DATE'] = Guber['DATE'].apply(lambda x: datetime(int(x[:4]), int(x[4:6]), int(x[6:])))


div=pd.read_table("Tarea1-AdRF/tasa_dividendos.txt")
div = div.rename(columns = {'DATE ':'DATE'})
div['DATE'] =div['DATE'].apply(lambda x: str(x))
div['DATE'] = div['DATE'].apply(lambda x: datetime(int(x[:4]), int(x[4:6]), int(x[6:])))
nombre = list(div.columns)
nombre.remove('DATE')
div[nombre] = div[nombre]/100

S = pd.read_table("Tarea1-AdRF/tasas_IPC.txt")
S['Close']= S['Close'].replace("N/E", np.NaN)
S= S.dropna()
S['Close'] = pd.to_numeric(S['Close'])
S = S.rename(columns = {'Date':'DATE','Close':'^MXX'})
S['DATE'] =S['DATE'].apply(lambda x: str(x))
S['DATE'] = S['DATE'].apply(lambda x: datetime(int(x[:4]), int(x[4:6]), int(x[6:])))

Y = pd.read_table("Tarea1-AdRF/tasa_spot.txt")
Y = Y.rename(columns = {'date':'DATE','precio':'USDMXN=X'})
Y['DATE'] =Y['DATE'].apply(lambda x: str(x))
Y['DATE'] = Y['DATE'].apply(lambda x: datetime(int(x[:4]), int(x[4:6]), int(x[6:])))

S = S.merge(Y, on = 'DATE',how='inner')

tx = [Guber,Guber]
ty=[Libor,div]


K = [20.83,49525]
M = [100,50]
vencimientos = [5,53]

In [5]:
miForward.compute(M,S,K,vencimientos,tx,ty,alisado=False)
miForward.precios

IndexError: ignored

## Sin Alisado

In [117]:
miForward.compute(M,S,K,vencimientos,tx,ty,alisado=False)
miForward.precios

IndexError: ignored

## Con Alisado

In [119]:
miForward.compute(M,S,K,vencimientos,tx,ty,alisado = True)

IndexError: ignored

# SWAP

Un swap largo con nocional de 16 millones pagando tasa fija de 6.6% anual (cada 28 días) y recibiendo tasa
flotante de la curva TIIE vencimiento de 588 días, un swap corto con nocional de 12 millones pagando tasa
variable de la TIIE y recibiendo fija de 5.9% vencimiento de 270 días.

In [120]:
def InterpolacionLineal(x,X,Y):
    n = len(X)
    for i in range(n-1):
        if X[i]<= x <= X[i+1]:
            a = Y[i+1] - Y[i]
            b = (X[i+1] - X[i]).days
            return Y[i] + (x - X[i]).days*(a/b)
    return "No es posible interpolar"

class InterestRateSwap:
    
    def __init__(self):
        self.precio = None
        self.n_cupones = None
        self.nocional = None
        self.posicion = None
        self.convencion = None
        self.fija = None
        self.flotante = None
        self.descuentos = None
        self.summary = None
        
    def compute(self, fval, M, n, fix_rate, flotante,calen_flotante, descuentos, calen_desc, 
                plazos, tau,  payer = False, act_360 = True):
        """
        M: Nocional
        n: # de cupones
        fix_rate: Tasa fija de la pata larga del IRS
        flotante: Curva de tasas de mercado (la que encuentras en BANXICO) de la pata corta
        calen_flotante: Las fechas dadas por la curva de flotante
        descuentos: Curca de tasas de descuento de mercado para valuar el IRS
        calen_desc: Las fechas dadas por la curva de descuentos
        plazos : Lista de plazo del i -ésimo cupón (habiles)
        tau: Plazo en dias del i-ésimo cupón. (naturales)
        payer: Dummy; 1 si paga fija, 0 si paga flotante
        act_360: Dummy; tipo de convención 1 si act/360, 0 si act/365
        """

        spot = fval
        
        if act_360:
            conv = 360
        else:
            conv = 365
            
        aux = pd.DataFrame({"Cupon":list(range(1,n+1)), "Tau":tau})
        aux["Desde Spot"] = aux["Tau"].cumsum()
        aux["Fecha"] = aux["Desde Spot"].apply(lambda x: timedelta(x) + spot)
                
        df = pd.DataFrame({"Cupon":list(range(1,n+1)), "Plazo": plazos, "Tau": tau})
                
        df["Fechas"] =  aux["Fecha"]
        df["Flotante"] = df["Fechas"].apply(InterpolacionLineal, args=(calen_flotante, flotante))
        df["Descuentos"]=df["Fechas"].apply(InterpolacionLineal, args=(calen_desc, descuentos))
        
        # Calculo de tasas forward
        df['Forward_Flotante'] = (conv/(aux['Desde Spot']-aux['Desde Spot'].shift(1)))*(((1+(df['Flotante']*aux['Desde Spot']/conv))/(1+(df['Flotante'].shift(1)*aux['Desde Spot'].shift(1)/conv))) - 1)
        df['Forward_Flotante'][0] = df['Flotante'][0]
        
        df["Sumando"] = ((df["Forward_Flotante"]-fix_rate)*df["Plazo"]/conv) / (1 + (df["Descuentos"]*df["Tau"]/conv))
        
        self.precio =  M*((-1)**payer)*df["Sumando"].sum()
        self.n_cupones = n
        self.nocional = M
        self.posicion = "Payer" if payer else "Receiver"
        self.convencion = conv
        self.fija = fix_rate
        self.flotante = df[["Fechas","Forward_Flotante"]]
        self.descuentos = df[["Fechas", "Descuentos"]]
        self.summary = df
        
        return
    
    def plots(self):
        Bool = True
        while Bool:
            print("\n")
            choice = input("Si deseas ver las tasas de descuento teclea 'Descuentos', si deseas\
                                ver las tasas flotantes teclea 'Tasas' o si deseas salir escribe '0':\n")
            if choice == 'Descuentos' or choice =='Tasas' or choice == '0':
                Bool = False
            else:
                print('\nTu elección no es válida')
        if choice == 'Descuentos':
            sns.set_style('darkgrid')
            sns.set_palette('tab10')
            plt.figure(figsize = (12,8))
            ax = sns.lineplot(x = self.summary['Fechas'], y = self.summary['Descuentos'])
            ax.set_title("Curva de Descuentos",fontsize = '25')
            plt.show()
            
            return
        elif choice == 'Tasas':
            sns.set_style('darkgrid')
            sns.set_palette('tab10')
            plt.figure(figsize = (12,8))
            ax = sns.lineplot(x = self.summary['Fechas'], y = self.summary['Forward_Flotante'], color = 'red')
            ax.set_title("Curva de Tasas Forward Flotantes",fontsize = '25')
            plt.show()
            
            return
        else:
            return

In [121]:
class Riesgo_IRS:

  def __init__(self):
    self.posiciones = None
    self.tasas_fijas = None
    self.vencimientos = None
    self.dias_bono = None
    self.VaR = None
    self.CVaR = None
    self.alpha = None
    self.summary = None
    self.ordenado = None
    self.alisado = None
    self.precios = None
    
  def compute(self, fval, tasas_fijas, cupon_l, descuento_l,cupon_c,descuento_c,\
              vencimientos, dias_bono, posiciones, alpha,alisado =False):
    """
    a
    """
    calen_flotante_l = cupon_l.columns
    calen_flotante_l = list(calen_flotante_l)
    calen_flotante_l.remove('DATE')
    calen_desc_l = descuento_l.columns
    calen_desc_l = list(calen_desc_l)
    calen_desc_l.remove('DATE')


    calen_flotante_c = cupon_c.columns
    calen_flotante_c = list(calen_flotante_c)
    calen_flotante_c.remove('DATE')
    calen_desc_c = descuento_c.columns
    calen_desc_c = list(calen_desc_c)
    calen_desc_c.remove('DATE')

    
    n_l = int(vencimientos[0]/dias_bono[0])
    if n_l == (vencimientos[0]/dias_bono[0]):
        plazos_l = [dias_bono[0]]*n_l
        tau_l = plazos_l
    else:
        plazos_l = [vencimientos[0]-n_l*dias_bono[0]] + [dias_bono[0]]*(n_l-1)
        tau_l = plazos_l

    n_c = int(vencimientos[1]/dias_bono[1])
    if n_c == (vencimientos[1]/dias_bono[1]):
        plazos_c = [dias_bono[1]]*n_c
        tau_c = plazos_c
    else:
        plazos_c = [vencimientos[1]-n_c*dias_bono[1]] + [dias_bono[1]]*(n_c-1)
        tau_c = plazos_c



    n_cupl = cupon_l.shape[1] - 1
    n_descl = descuento_l.shape[1] - 1
    n_cupc = cupon_c.shape[1] - 1
    n_descc = descuento_c.shape[1] - 1

    # 
    X = cupon_l.merge(descuento_l,how = 'inner',on = 'DATE')
    X = X.merge(cupon_c,how = 'inner',on = 'DATE')
    X = X.merge(descuento_c,how = 'inner',on = 'DATE')
    X.set_index('DATE', inplace=True)
    X0 = X.head(1)
    DeltaX = (X.shift(1)/X)-1
    DeltaX.iloc[0,:] = 0
    X_s = X0.values*(1+DeltaX)

    Y_s = pd.DataFrame()

    cup_l = X_s.iloc[:,:n_cupl]
    desc_l = X_s.iloc[:,n_cupl:n_cupl+n_descl]
    cup_c = X_s.iloc[:,n_cupl+n_descl:n_cupl+n_descl+n_cupc]
    desc_c = X_s.iloc[:,n_cupl+n_descl+n_cupc:n_cupl+n_descl+n_cupc+n_descc]

    # Largo
    largo = []
    for i in range(len(cup_l)):
      fval_aux = cup_l.index[i]
      flotante =  cup_l.iloc[i,:].values/100
      flotante = list(flotante)
      descuentos = desc_l.iloc[i,:].values/100
      descuentos = list(descuentos)
      calen_flotante_l_aux = [fval_aux + timedelta(int(i)) for i in calen_flotante_l]
      calen_desc_l_aux = [fval_aux + timedelta(int(i)) for i in calen_desc_l]
      irs = InterestRateSwap()
      # fval vs fval_aux
      irs.compute(fval_aux, posiciones[0], n_l,tasas_fijas[0] , flotante,calen_flotante_l_aux, descuentos, calen_desc_l_aux, plazos_l, tau_l)
      largo.append(irs.precio)

    # Corto
    corto = []
    for i in range(len(cup_c)):
      fval_aux = cup_c.index[i]
      flotante =  cup_c.iloc[i,:].values/100
      flotante = list(flotante)
      descuentos = desc_c.iloc[i,:].values/100
      descuentos = list(descuentos)
      calen_flotante_c_aux = [fval_aux + timedelta(int(i)) for i in calen_flotante_c]
      calen_desc_c_aux = [fval_aux + timedelta(int(i)) for i in calen_desc_c]
      irs = InterestRateSwap()
      # fval vs fval_aux
      irs.compute(fval_aux, posiciones[1], n_c, tasas_fijas[1] , flotante,calen_flotante_c_aux, descuentos, calen_desc_c_aux, plazos_c, tau_c)
      corto.append(irs.precio)

    Y_s['IRS_l'] = largo
    Y_s['IRS_c'] = corto
    #print(Y_s)
    posiciones = pd.Series(posiciones,index = ['IRS_l','IRS_c'])
    Y0 = Y_s.head(1)
    Y0 = Y0.values[0]
    self.precios = Y0
    #print(Y0)
    PL = (Y_s-Y0)*posiciones
    PL['Total']=PL.sum(axis=1)
    #print(PL)
    VaR_C=[]

    if alisado:
      c = 1
      PL.dropna(inplace = True)
      #PL = PL.iloc[::-1]
      PL["Esc_SA"] = ExponentialWeighted(0.05,PL.shape[0])
      col_aux = list(PL.columns)
      col_aux.remove("Esc_SA")
      vector_VaR = [VaR_CA(PL.iloc[:,i],PL.Esc_SA,alpha) for i in range(PL.shape[1]-1)]
      VaR = pd.Series(vector_VaR,index = col_aux)
    else:
      c = 0
      VaR=PL.quantile(alpha, numeric_only=False)


    for i in range(0,PL.shape[1]-c):
      VC=CVaR(PL.iloc[:,i],VaR[i])
      VaR_C.append(VC)
    
    self.summary = pd.DataFrame({"VaR":VaR})
    self.summary['CVaR'] = VaR_C

    self.posiciones = {'Largo':posiciones[0],'Corto':posiciones[1]}
    self.tasas_fijas = {'Largo':tasas_fijas[0],'Corto':tasas_fijas[1]}
    self.vencimientos = {'Largo':vencimientos[0],'Corto':vencimientos[1]}
    self.dias_bono = {'Largo':dias_bono[0],'Corto':dias_bono[1]}
    self.VaR = self.summary[['VaR']]
    self.CVaR = self.summary[['CVaR']]
    self.alpha = alpha
    self.alisado = alisado
    self.precios = Y0
    return 


In [129]:
# SWAP para el caso con CUPÓN
fval = datetime(2023,3,10) # Fecha de valoración: 10 de marzo de 2023
tasas_fijas = [0.066,0.059]  # SWAP Largo tasa fija del 6.6% anual
                             # Recibe tasa fija de 5.9%


cupon_l = pd.read_csv("Tarea1-AdRF/tasa_DIRS_SW_OP.txt", delimiter = "\t")
cupon_l = cupon_l.rename(columns = {'DATE ':'DATE'})
cupon_l['DATE'] =cupon_l['DATE'].apply(lambda x: str(x))
cupon_l['DATE'] = cupon_l['DATE'].apply(lambda x: datetime(int(x[:4]), int(x[4:6]), int(x[6:])))
cupon_c  = cupon_l

descuento_l = pd.read_csv("Tarea1-AdRF/tasa_TIIE_SW_OP.txt", delimiter = "\t")
descuento_l = descuento_l.rename(columns = {'DATE ':'DATE'})
descuento_l['DATE'] =descuento_l['DATE'].apply(lambda x: str(x))
descuento_l['DATE'] = descuento_l['DATE'].apply(lambda x: datetime(int(x[:4]), int(x[4:6]), int(x[6:])))
descuento_c = descuento_l


vencimientos = [588,270] # Un SWAP vence en 588 días y el otro en 270 días
dias_bono = [28,28]      # Días del bono
posiciones  = [16000000, -12000000] # Valor del nocional por cada SWAP
alpha = 0.98

In [130]:
miIRS= Riesgo_IRS()

## Sin Alisado

In [ ]:
miIRS.compute(fval, tasas_fijas, cupon_l, descuento_l,cupon_c,descuento_c,\
              vencimientos, dias_bono, posiciones, alpha,False)

In [132]:
miIRS.summary

,VaR,CVaR
IRS_l,3.061388e+11,5.126185e+11
IRS_c,7.601928e+10,1.208232e+11
Total,3.283307e+11,5.712966e+11


## Con Alisado

In [ ]:
miIRS.compute(fval, tasas_fijas, cupon_l, descuento_l,cupon_c,descuento_c,\
              vencimientos, dias_bono, posiciones, alpha,True)

In [134]:
miIRS.summary

,VaR,CVaR
IRS_l,1.970513e+11,3.036477e+11
IRS_c,4.490584e+10,7.785653e+10
Total,2.102452e+11,3.391953e+11


Obtenemos el valor de las posiciones.

In [138]:
print(miIRS.posiciones)
print(miIRS.tasas_fijas)
print(miIRS.vencimientos)
print(miIRS.dias_bono)
print(miIRS.precios)

{'Largo': 16000000, 'Corto': -12000000}
{'Largo': 0.066, 'Corto': 0.059}
{'Largo': 588, 'Corto': 270}
{'Largo': 28, 'Corto': 28}
[ 281990.62710715 -173029.84047835]


# Opciones

Dos opciones europeas, una de tasa de interés larga call con strike de 5.8 % vencimiento de 1700 días 1000
contratos, y otra put larga de tasa de interés con strike de 6.0 % de la tiie nocional de 500 contratos vencimiento 700 días. (se valuará basado en las superficies de volatilidad si es in o out the money y en las curvas de tasa de interés TIIE o Pagarés).

In [ ]:
%%R
#require(quantmod)
install.packages("quantmod")
library(quantmod)
# require(data.table)
install.packages("data.table")
library(data.table)

install.packages("Deriv")
library(Deriv)


library(dplyr)
con = gzcon(url('https://github.com/systematicinvestor/SIT/raw/master/sit.gz', 'rb'))
source(con)
close(con)

In [140]:
%%R
#FUNCIÓN DE INTERPOLACIÓN ALAMBRADA

talamb=function(nodos,curva,plazos) #función de interpolación de tasas por el método alamabrada
{
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n)
  {
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}

#funciones necesarias
diagv=function(x)			#función para diagonalizar un vector
{
  n01=nrow(as.matrix(x))
  m01=ncol(as.matrix(x))
  dimmax=max(n01,m01)
  res=matrix(0,dimmax,dimmax)
  for (i in 1:dimmax)
  {
    res[i,i]=x[i]
  }
  res
}


diagm=function(x)		#función para diagonalizar una matriz en una de mayor dimensión
{
  n01=nrow(as.matrix(x))
  m01=ncol(as.matrix(x))
  if(m01>=n01 )
  {
    res=matrix(0,m01,m01)
    for (i in 1:(m01/n01))
    {
      z=(i-1)*n01
      for(j in 1:n01)
      {
        for(k in 1:n01)
        {
          res[z+k,z+j]=x[k,z+j]
        }
      }
    }
    res
  }
  else
  {
    "no es matriz cuadrada"
  }
}

#Función de cuantil más cercano
equantile <- function(v,p=.5,ns=nrow(as.matrix(v))) 
{ 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  ranking <- order(v) 
  vw=matrix(0,ns,1)
  vw[1:ns]=seq(1/ns,ns)  
  sumw <- cumsum(vw[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
} 

In [141]:
%%R

#MEDIDAS DE RIESGO CON ALISADO

#VaR con alisado
wquantile <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
} 

#CVaR con alisado
wcvar <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  loss= v [ ranking [ which( plist <= p ) ] ]  
  esc=w [ ranking [ which( plist <= p ) ] ]  
  sum(loss*esc)/(sum(esc))
} 


In [142]:
# Definción de parámetros para valorar:

%%R
fval=as.Date("20230310",format="%Y%m%d") #Fecha de valoración
itpl=0 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada
alpha=0.98 #Nivel de confianza para obtener estimaciones de riesgo
nh=3660 #número de días de historia
#yext=1 #si se usa la historia de internet o fija

btasadesc_oir="Tarea1-AdRF/tasa_TIIE_SW_OP.txt"
btasaspot_oir="Tarea1-AdRF/tasa_DIRS_SW_OP.txt"
bvolspot_oir="Tarea1-AdRF/tvoltiie_opc.txt"
plazos_oir=cbind(1700, 700) #T-t
pr_oir=28 #plazo de referencia
dct_oir=360 #d_base
cp_oir=cbind(1,0) #si es call (cap) o put (floor)
K_oir=cbind( 0.058, 0.06)
contratos_oir=cbind(1000, 500)
nominal_oir=1 
cs_oir=1 #1 si es continua la tasa 0 si es simple

In [143]:
%%R

#CARGA DE DATOS PARA OPCIONES DE TASA DE INTERÉS
  #carga de datos
  #carga de rho	
  data1<-read.table(btasadesc_oir) 
  n<-nrow(data1)
  m1_orig_oir=ncol(data1)
  x1_orig_oir=data.table(mutate(data1[2:n,1:m1_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x1_orig_oir=x1_orig_oir%>%select(-V1)
  nodos1_oir=data.frame(data1[1,2:m1_orig_oir])
  
  #data1[1:3,]
  #carga de tasas spot
  data2<-read.table(btasaspot_oir)
  n<-nrow(data2)
  m2_orig_oir=ncol(data2)
  x2_orig_oir=data.table(mutate(data2[2:n,1:m2_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x2_orig_oir=x2_orig_oir%>%select(-V1)
  nodos2_oir=data.frame(data2[1,2:m2_orig_oir])
  #carga de volatilidades de spot
  data3<-read.table(bvolspot_oir)
  n<-nrow(data3)
  m3_orig_oir=ncol(data3)
  x3_orig_oir=data.table(mutate(data3[2:n,1:m3_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x3_orig_oir=x3_orig_oir%>%select(-V1)
  
  nodos3_oir=data.frame(data3[1,2:m3_orig_oir])

In [144]:
%%R
#INTERSECCIÓN DE FECHAS DE LAS OPCIONES    
  
lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp=data.table(Date=as.Date(x1_orig_oir[x2_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot, opc ir vp
lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp[x3_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot, opc ir vp, opc ir vol


In [145]:
%%R
head(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp)
head(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol)


         Date
1: 2023-02-28
2: 2023-02-27
3: 2023-02-24
4: 2023-02-23
5: 2023-02-22
6: 2023-02-21


In [146]:
%%R
n=nrow(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol) #Historia de todos

#historia de opciones
x1_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x1_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x1_orig_oir=x1_orig_oir%>%select(-Date)/100
x2_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x2_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x2_orig_oir=x2_orig_oir%>%select(-Date)/100
x3_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x3_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x3_orig_oir=x3_orig_oir%>%select(-Date)

In [147]:
%%R


##opciones de tasa de interés, con inicio el día de la valuación CÁLCULO
#Posición inicial

#interpolación de tasas y volatilidades
m=ncol(plazos_oir)
x1=matrix(0,n,m)
x2tc=matrix(0,n,m)
x2tl=matrix(0,n,m)
x2=matrix(0,n,m)
x3=matrix(0,n,m)
for (i in 1:(n))
{
  x1[i,]=if(itpl==0){approx(nodos1_oir,x1_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos1_oir,x1_orig_oir[i,],plazos_oir)}
  x2tc[i,]=if(itpl==0){approx(nodos2_oir,x2_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos2_oir,x2_orig_oir[i,],plazos_oir)}
  x2tl[i,]=if(itpl==0){approx(nodos2_oir,x2_orig_oir[i,],(plazos_oir+pr_oir),rule=2)$y}else{talamb(nodos2_oir,x2_orig_oir[i,],(plazos_oir+pr_oir))}
  x3[i,]=if(itpl==0){approx(nodos3_oir,x3_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos3_oir,x3_orig_oir[i,],plazos_oir)}
  x2[i,]=((1+x2tl[i,]*(plazos_oir+pr_oir)/360)/(1+x2tc[i,]*(plazos_oir)/360)-1)*360/pr_oir
}

#Se toma la primera fila de las tasas, ie la tasa más actual para calcular la valoracion
x01=x1[1,] #tasas de descuento
x02=x2[1,] #tasas spot
x03=x3[1,] #volatilidades

X_oir=cbind(x1,x2,x3)

opctint = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#función de una opción europea
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}  
  (if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir}else{(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360))*(-1)^cp_oir})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)
}

#Usa la funcion para valuar las opciones
V0_oir=opctint(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio
V0_oir


          [,1]      [,2]
[1,] 0.7591073 0.4324731


In [148]:
%%R
#DIMENSION DE TODOS LOS INSTRUMENTOS
n_if=matrix(0,1,1)
n_if[1]=ncol(X_oir) #fwd tdc

#valor del portafolio

V0_port=cbind(V0_oir)
V0T_port=sum(V0_port)


#INTEGRACIÓN DE TODOS LOS FACTORES DE RIESGO EN UNA MATRIZ
X_port=cbind(X_oir) #Factores de riesgo del portafolios de 2 opciones y 3 factores de riesgo

#Tasas que nos interesan 
#print(head(X_port))


#Cálculo de variaciones Delta_X DEL PORTAFOLIOS
DeltaX_port=as.matrix(log(X_port[1:(n-1),]/X_port[2:(n),]))
DeltaX_port[is.nan(DeltaX_port)] <- 0 #quitamos NaN
DeltaX_port[is.na(DeltaX_port)] <- 0 #quitamos Na
DeltaX_port[is.infinite(DeltaX_port)] <- 0 #quitamos Na

Ns=nrow(DeltaX_port) #Definimos número de escenarios históricos
DeltaX_s=DeltaX_port

##  Sin Alisado

In [150]:
%%R
m=ncol(plazos_oir) 
X_s_oir=matrix(0,Ns,n_if[1]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs) #PASO CLAVE
V_oir=matrix(0,Ns,m)
Vfr1_oir=matrix(0,Ns,m)
Vfr2_oir=matrix(0,Ns,m)
Vfr3_oir=matrix(0,Ns,m)
PG_oir=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_oir=matrix(0,Ns,m)
PGfr2_oir=matrix(0,Ns,m)
PGfr3_oir=matrix(0,Ns,m)
PGT_oir=matrix(0,Ns,1)
PGfr1T_oir=matrix(0,Ns,1)
PGfr2T_oir=matrix(0,Ns,1)
PGfr3T_oir=matrix(0,Ns,1)

DeltaX_s_oir=DeltaX_s  
x0_oir=X_oir[1,] 

for (i in 1:Ns)
{
  X_s_oir[i,]=x0_oir*exp(DeltaX_s_oir[i,])
  #PASO CLAVE
  V_oir[i,]=  opctint(X_s_oir[i,(1:(n_if[1]/3))],X_s_oir[i,((n_if[1]/3+1):(n_if[1]/3*2))],K_oir,X_s_oir[i,((n_if[1]*2/3+1):(n_if[1]))],plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio  
  #PASO CLAVE
  Vfr1_oir[i,]=opctint(X_s_oir[i,(1:(n_if[1]/3))],x0_oir[((n_if[1]/3+1):(n_if[1]/3*2))],K_oir,x0_oir[((n_if[1]*2/3+1):(n_if[1]))],plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio  
  #PASO CLAVE
  Vfr2_oir[i,]=opctint(x0_oir[(1:(n_if[1]/3))],X_s_oir[i,((n_if[1]/3+1):(n_if[1]/3*2))],K_oir,x0_oir[((n_if[1]*2/3+1):(n_if[1]))],plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio  
  #PASO CLAVE
  Vfr3_oir[i,]=opctint(x0_oir[(1:(n_if[1]/3))],x0_oir[((n_if[1]/3+1):(n_if[1]/3*2))],K_oir,X_s_oir[i,((n_if[1]*2/3+1):(n_if[1]))],plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio  
  PG_oir[i,]=V_oir[i,]-V0_oir
  PGfr1_oir[i,]=Vfr1_oir[i,]-V0_oir
  PGfr2_oir[i,]=Vfr2_oir[i,]-V0_oir
  PGfr3_oir[i,]=Vfr3_oir[i,]-V0_oir
  PGT_oir[i,]=sum(PG_oir[i,])
  PGfr1T_oir[i,]=sum(PGfr1_oir[i,])
  PGfr2T_oir[i,]=sum(PGfr2_oir[i,])
  PGfr3T_oir[i,]=sum(PGfr3_oir[i,])
}



#VaR por posición
VaRCont_oir=matrix(0,1,m)
VaRfr1_oir=matrix(0,1,m)
VaRfr2_oir=matrix(0,1,m)
VaRfr3_oir=matrix(0,1,m)
CVaRCont_oir=matrix(0,1,m)
CVaRfr1_oir=matrix(0,1,m)
CVaRfr2_oir=matrix(0,1,m)
CVaRfr3_oir=matrix(0,1,m)
for (i in (1:m))
{
  VaRCont_oir[i]=equantile(PG_oir[,i],1-alpha,Ns)
  VaRfr1_oir[i]=equantile(PGfr1_oir[,i],1-alpha,Ns)
  VaRfr2_oir[i]=equantile(PGfr2_oir[,i],1-alpha,Ns)
  VaRfr3_oir[i]=equantile(PGfr3_oir[,i],1-alpha,Ns)
  CVaRfr1_oir[i]= mean(merge(which(PGfr1_oir[,i]<VaRfr1_oir[i]),cbind(seq(1,Ns),PGfr1_oir[,i]), by.x=1,by.y=1)[,2])
  CVaRfr2_oir[i]= mean(merge(which(PGfr2_oir[,i]<VaRfr2_oir[i]),cbind(seq(1,Ns),PGfr2_oir[,i]), by.x=1,by.y=1)[,2])
  CVaRfr3_oir[i]= mean(merge(which(PGfr3_oir[,i]<VaRfr3_oir[i]),cbind(seq(1,Ns),PGfr3_oir[,i]), by.x=1,by.y=1)[,2])
  CVaRCont_oir[i]= mean(merge(which(PG_oir[,i]<VaRCont_oir[i]),cbind(seq(1,Ns),PG_oir[,i]), by.x=1,by.y=1)[,2])
}


#VaR Total
VaRTotal_oir=equantile(PGT_oir,1-alpha,Ns)
CVaRTotal_oir= mean(merge(which(PGT_oir<VaRTotal_oir),cbind(seq(1,Ns),PGT_oir), by.x=1,by.y=1)[,2])
VaRTotalfr1_oir=equantile(PGfr1T_oir,1-alpha,Ns)
CVaRTotalfr1_oir= mean(PGfr1T_oir[which(PGfr1T_oir<VaRTotalfr1_oir),])
VaRTotalfr2_oir=equantile(PGfr2T_oir,1-alpha,Ns)
CVaRTotalfr2_oir= mean(PGfr2T_oir[which(PGfr2T_oir<VaRTotalfr2_oir),])
VaRTotalfr3_oir=equantile(PGfr3T_oir,1-alpha,Ns)
CVaRTotalfr3_oir= mean(PGfr3T_oir[which(PGfr2T_oir<VaRTotalfr2_oir),])


print("Valor del portafolio")
print(V0_oir)
print("VaR por contrato")
print(VaRCont_oir)
print("VaR por el factor de riesgo tasas de descuento por contrato")
print(VaRfr1_oir)
print("VaR por el factor de riesgo tasas spot por contrato")
print(VaRfr2_oir)
print("VaR por el factor de riesgo volatilidades por contrato")
print(VaRfr3_oir)
print("VaR por el factor de riesgo tasas de descuento total")
print(VaRTotalfr1_oir)
print("VaR por el factor de riesgo tasas spot 2 total")
print(VaRTotalfr2_oir)
print("VaR por el factor de riesgo volatilidades total")
print(VaRTotalfr3_oir)
print("VaR Total")
print(VaRTotal_oir)

print("CVaR por contrato")
print(CVaRCont_oir)
print("CVaR por el factor de riesgo tasas de descuento por contrato")
print(CVaRfr1_oir)
print("CVaR por el factor de riesgo tasas spot por contrato")
print(CVaRfr2_oir)
print("CVaR por el factor de riesgo volatilidades por contrato")
print(CVaRfr3_oir)
print("CVaR por el factor de riesgo tasas de descuento total")
print(CVaRTotalfr1_oir)
print("CVaR por el factor de riesgo tasas spot total")
print(CVaRTotalfr2_oir)
print("CVaR por el factor de riesgo volatilidades total")
print(CVaRTotalfr3_oir)
print("CVaR Total")
print(CVaRTotal_oir)


[1] "Valor del portafolio"
          [,1]      [,2]
[1,] 0.7591073 0.4324731
[1] "VaR por contrato"
            [,1]        [,2]
[1,] -0.06650712 -0.05306331
[1] "VaR por el factor de riesgo tasas de descuento por contrato"
             [,1]          [,2]
[1,] -0.005392273 -0.0009158142
[1] "VaR por el factor de riesgo tasas spot por contrato"
            [,1]        [,2]
[1,] -0.05332608 -0.04557009
[1] "VaR por el factor de riesgo volatilidades por contrato"
            [,1]        [,2]
[1,] -0.03410272 -0.02882067
[1] "VaR por el factor de riesgo tasas de descuento total"
[1] -0.006410024
[1] "VaR por el factor de riesgo tasas spot 2 total"
[1] -0.06062326
[1] "VaR por el factor de riesgo volatilidades total"
[1] -0.05654009
[1] "VaR Total"
[1] -0.07728335
[1] "CVaR por contrato"
           [,1]        [,2]
[1,] -0.1110112 -0.08302614
[1] "CVaR por el factor de riesgo tasas de descuento por contrato"
             [,1]         [,2]
[1,] -0.005704847 -0.001742734
[1] "CVaR por el fact

## Con Alisado

In [151]:
%%R
w0 <- 0.05
lambda =uniroot(function(x) w0*(1-x^(n))/(1-x)-1, c(0,0.99), tol = 1e-28)$root


#generamos la función que genera "n" escenarios con base en w0 y lambda
genera_esc=function(lamda,w0,n)
{
  p_esc=matrix(0,n,1)
  for (i in (1:n))
  {
    p_esc[i]=w0*lambda^(i-1)
  }
  p_esc
}

p_esc=genera_esc(lambda,w0,Ns)

In [152]:
%%R
m=ncol(plazos_oir)
VaRCACont_oir=matrix(0,1,m)
VaRCAfr1_oir=matrix(0,1,m)
VaRCAfr2_oir=matrix(0,1,m)
VaRCAfr3_oir=matrix(0,1,m)
CVaRCACont_oir=matrix(0,1,m)
CVaRCAfr1_oir=matrix(0,1,m)
CVaRCAfr2_oir=matrix(0,1,m)
CVaRCAfr3_oir=matrix(0,1,m)
for (i in (1:m))
{
  VaRCACont_oir[i]=wquantile(PG_oir[,i],p_esc,1-alpha)
  VaRCAfr1_oir[i]=wquantile(PGfr1_oir[,i],p_esc,1-alpha)
  VaRCAfr2_oir[i]=wquantile(PGfr2_oir[,i],p_esc,1-alpha)
  VaRCAfr3_oir[i]=wquantile(PGfr3_oir[,i],p_esc,1-alpha)
  CVaRCAfr1_oir[i]= wcvar(PGfr1_oir[,i],p_esc,1-alpha)
  CVaRCAfr2_oir[i]= wcvar(PGfr2_oir[,i],p_esc,1-alpha)
  CVaRCAfr3_oir[i]= wcvar(PGfr3_oir[,i],p_esc,1-alpha)
  CVaRCACont_oir[i]= wcvar(PG_oir[,i],p_esc,1-alpha)
}


#VaRCA Total
VaRCATotal_oir=wquantile(PGT_oir,p_esc,1-alpha)
CVaRCATotal_oir= wcvar(PGT_oir,p_esc,1-alpha)
VaRCATotalfr1_oir=wquantile(PGfr1T_oir,p_esc,1-alpha)
CVaRCATotalfr1_oir= wcvar(PGfr1T_oir,p_esc,1-alpha)
VaRCATotalfr2_oir=wquantile(PGfr2T_oir,p_esc,1-alpha)
CVaRCATotalfr2_oir= wcvar(PGfr2T_oir,p_esc,1-alpha)
VaRCATotalfr3_oir=wquantile(PGfr3T_oir,p_esc,1-alpha)
CVaRCATotalfr3_oir= wcvar(PGfr3T_oir,p_esc,1-alpha)

print("Valor del portafolio")
print(V0_oir)
print("VaRCA por contrato")
print(VaRCACont_oir)
print("VaRCA por el factor de riesgo tasas de descuento por contrato")
print(VaRCAfr1_oir)
print("VaRCA por el factor de riesgo tasas spot por contrato")
print(VaRCAfr2_oir)
print("VaRCA por el factor de riesgo volatilidades por contrato")
print(VaRCAfr3_oir)
print("VaRCA por el factor de riesgo tasas de descuento total")
print(VaRCATotalfr1_oir)
print("VaRCA por el factor de riesgo tasas spot total")
print(VaRCATotalfr2_oir)
print("VaRCA por el factor de riesgo volatilidades total")
print(VaRCATotalfr3_oir)
print("VaRCA Total")
print(VaRCATotal_oir)

print("CVaRCA por contrato")
print(CVaRCACont_oir)
print("CVaRCA por el factor de riesgo tasas de descuento por contrato")
print(CVaRCAfr1_oir)
print("CVaRCA por el factor de riesgo tasas spot por contrato")
print(CVaRCAfr2_oir)
print("CVaRCA por el factor de riesgo volatilidades por contrato")
print(CVaRCAfr3_oir)
print("CVaRCA por el factor de riesgo tasas de descuento total")
print(CVaRCATotalfr1_oir)
print("CVaRCA por el factor de riesgo tasas spot total")
print(CVaRCATotalfr2_oir)
print("CVaRCA por el factor de riesgo volatilidades total")
print(CVaRCATotalfr3_oir)
print("CVaRCA Total")
print(CVaRCATotal_oir)

[1] "Valor del portafolio"
          [,1]      [,2]
[1,] 0.7591073 0.4324731
[1] "VaRCA por contrato"
            [,1]        [,2]
[1,] -0.04960231 -0.03164635
[1] "VaRCA por el factor de riesgo tasas de descuento por contrato"
             [,1]          [,2]
[1,] -0.003179866 -0.0005385752
[1] "VaRCA por el factor de riesgo tasas spot por contrato"
            [,1]        [,2]
[1,] -0.04392037 -0.03161865
[1] "VaRCA por el factor de riesgo volatilidades por contrato"
            [,1]        [,2]
[1,] -0.03465482 -0.02155953
[1] "VaRCA por el factor de riesgo tasas de descuento total"
[1] -0.003448647
[1] "VaRCA por el factor de riesgo tasas spot total"
[1] -0.03900403
[1] "VaRCA por el factor de riesgo volatilidades total"
[1] -0.04751759
[1] "VaRCA Total"
[1] -0.06252591
[1] "CVaRCA por contrato"
            [,1]        [,2]
[1,] -0.06760054 -0.04457509
[1] "CVaRCA por el factor de riesgo tasas de descuento por contrato"
             [,1]          [,2]
[1,] -0.004164656 -0.0006644505

# Ejercicio 2

2. Crear un programa en google colab (R o Python) para valorar portafolio de CCS peso mexicano - dólar tasa variable vs variable, y que calcule el riesgo por simulación histórica (2.5 puntos extra).

In [ ]:
class CrossCurrencySwap:
    
    def __init__(self):
        self.precio = None
        self.nocional_extranjera = None
        self.nocional_local = None
        self.n_cupones = None
        self.posicion = None
        self.convencion = None
        self.flotante_ext = None
        self.flotante_loc = None
        self.descuentos_ext = None
        self.descuentos_loc = None
        self.spot = None
        self.summary = None
        
    def compute(self, fval, n,M_l, M_e, Spot, flot_l,cal_flot_l, flot_e,cal_flot_e, desc_l, cal_desc_l,
                desc_e, cal_desc_e,plazos, tau,  payer = False, act_360 = True):
        """
        M: Nocional
        n: # de cupones
        fix_rate: Tasa fija de la pata larga del IRS
        flotante: Curva de tasas de mercado (la que encuentras en BANXICO) de la pata corta
        calen_flotante: Las fechas dadas por la curva de flotante
        descuentos: Curca de tasas de descuento de mercado para valuar el IRS
        calen_desc: Las fechas dadas por la curva de descuentos
        plazos : Lista de plazo del i -ésimo cupón (habiles)
        tau: Plazo en dias del i-ésimo cupón. (naturales)
        payer: Dummy; 1 si paga fija, 0 si paga flotante
        act_360: Dummy; tipo de convención 1 si act/360, 0 si act/365
        """

        spot = fval
        
        if act_360:
            conv = 360
        else:
            conv = 365
            
        aux = pd.DataFrame({"Cupon":list(range(1,n+1)), "Tau":tau})
        aux["Desde Spot"] = aux["Tau"].cumsum()
        aux["Fecha"] = aux["Desde Spot"].apply(lambda x: timedelta(x) + spot)
                
        df = pd.DataFrame({"Cupon":list(range(1,n+1)), "Plazo": plazos, "Tau": tau})
                
        df["Fechas"] =  aux["Fecha"]
        df["Flotante_Local"] = df["Fechas"].apply(InterpolacionLineal, args=(cal_flot_l, flot_l))
        df["Descuentos_Local"]=df["Fechas"].apply(InterpolacionLineal, args=(cal_desc_l, desc_l))
        df["Flotante_Ext"] = df["Fechas"].apply(InterpolacionLineal, args=(cal_flot_e, flot_e))
        df["Descuentos_Ext"]=df["Fechas"].apply(InterpolacionLineal, args=(cal_desc_e, desc_e))
        
        # Calculo de tasas forward
        df['Forward_Flotante_Local'] = (conv/(aux['Desde Spot']-aux['Desde Spot'].shift(1)))*(((1+(df['Flotante_Local']*aux['Desde Spot']/conv))/(1+(df['Flotante_Local'].shift(1)*aux['Desde Spot'].shift(1)/conv))) - 1)
        df['Forward_Flotante_Local'][0] = df['Flotante_Local'][0]
        df['Forward_Flotante_Ext'] = (conv/(aux['Desde Spot']-aux['Desde Spot'].shift(1)))*(((1+(df['Flotante_Ext']*aux['Desde Spot']/conv))/(1+(df['Flotante_Ext'].shift(1)*aux['Desde Spot'].shift(1)/conv))) - 1)
        df['Forward_Flotante_Ext'][0] = df['Flotante_Ext'][0]
        
        df["Sumando_Local"] = (M_l*((df["Forward_Flotante_Local"])*df["Plazo"]/conv)) / (1 + (df["Descuentos_Local"]*df["Tau"]/conv))
        df["Sumando_Ext"] = (M_e*Spot*((df["Forward_Flotante_Ext"])*df["Plazo"]/conv)) / (1 + (df["Descuentos_Ext"]*df["Tau"]/conv))
        df["Sumando"] = df["Sumando_Local"] - df["Sumando_Ext"]

        self.precio =  ((-1)**payer)*df["Sumando"].sum()
        self.nocional_extranjera = M_e
        self.nocional_local = M_l
        self.n_cupones = n
        self.posicion = "Payer" if payer else "Receiver"
        self.convencion = conv
        self.flotante_ext = df[["Fechas","Forward_Flotante_Ext"]]
        self.flotante_loc = df[["Fechas","Forward_Flotante_Local"]]
        self.descuentos_ext = df[["Fechas", "Descuentos_Ext"]]
        self.descuentos_loc = df[["Fechas", "Descuentos_Local"]]
        self.spot = Spot
        self.summary = df
        return
    
    # PENDIENTE
    def plots(self):
        Bool = True
        while Bool:
            print("\n")
            choice = input("Si deseas ver las tasas de descuento teclea 'Descuentos', si deseas\
                                ver las tasas flotantes teclea 'Tasas' o si deseas salir escribe '0':\n")
            if choice == 'Descuentos' or choice =='Tasas' or choice == '0':
                Bool = False
            else:
                print('\nTu elección no es válida')
        if choice == 'Descuentos':
            sns.set_style('darkgrid')
            sns.set_palette('tab10')
            plt.figure(figsize = (12,8))
            ax = sns.lineplot(x = self.summary['Fechas'], y = self.summary['Descuentos'])
            ax.set_title("Curva de Descuentos",fontsize = '25')
            plt.show()
            
            return
        elif choice == 'Tasas':
            sns.set_style('darkgrid')
            sns.set_palette('tab10')
            plt.figure(figsize = (12,8))
            ax = sns.lineplot(x = self.summary['Fechas'], y = self.summary['Forward_Flotante'], color = 'red')
            ax.set_title("Curva de Tasas Forward Flotantes",fontsize = '25')
            plt.show()
            
            return
        else:
            return

In [ ]:
class Riesgo_CCS:

  def __init__(self):
    self.nocionales = None
    self.vencimiento = None
    self.dias_bono = None
    self.VaR = None
    self.CVaR = None
    self.alpha = None
    self.spot = None
    self.summary = None
    self.ordenado = None
    self.alisado = None
    self.precios = None
    
  def compute(self, fval, spot,cupon_l, descuento_l,cupon_e,descuento_e, vencimiento,\
              dias_bono, posiciones, alpha, alisado = False):
    """
    a
    """
    calen_flotante_l = cupon_l.columns
    calen_flotante_l = list(calen_flotante_l)
    calen_flotante_l.remove('DATE')
    calen_desc_l = descuento_l.columns
    calen_desc_l = list(calen_desc_l)
    calen_desc_l.remove('DATE')


    calen_flotante_e = cupon_e.columns
    calen_flotante_e = list(calen_flotante_e)
    calen_flotante_e.remove('DATE')
    calen_desc_e = descuento_e.columns
    calen_desc_e = list(calen_desc_e)
    calen_desc_e.remove('DATE')


    n = int(vencimiento/dias_bono)
    if n == (vencimiento/dias_bono):
        plazos = [dias_bono]*n
        tau = plazos
    else:
        plazos = [vencimiento-n*dias_bono] + [dias_bono]*(n-1)
        tau = plazos

    # Aquí vamos
    n_cupl = cupon_l.shape[1] - 1
    n_descl = descuento_l.shape[1] - 1
    n_cupe = cupon_e.shape[1] - 1
    n_desce = descuento_e.shape[1] - 1

    # Spot
    cross_currency = yf.download(spot, end = fval)
    cross_currency = cross_currency[['Close']]
    cross_currency = cross_currency.iloc[::-1]
    cross_currency =cross_currency.reset_index()
    cross_currency = cross_currency.rename(columns = {'Date':'DATE','Close':spot})
    
    #  X

    X = cupon_l.merge(descuento_l,how = 'inner',on = 'DATE')
    X = X.merge(cupon_e,how = 'inner',on = 'DATE')
    X = X.merge(descuento_e,how = 'inner',on = 'DATE')
    X = X.merge(cross_currency, how = 'inner',on = 'DATE')
    X.set_index('DATE', inplace=True)

    X0 = X.head(1)
    DeltaX = (X.shift(1)/X)-1
    DeltaX.iloc[0,:] = 0
    X_s = X0.values*(1+DeltaX)



    Y_s = pd.DataFrame()
    irs_c= InterestRateSwap()
    cup_l = X_s.iloc[:,:n_cupl]
    desc_l = X_s.iloc[:,n_cupl:n_cupl+n_descl]
    cup_e = X_s.iloc[:,n_cupl+n_descl:n_cupl+n_descl+n_cupe]
    desc_e = X_s.iloc[:,n_cupl+n_descl+n_cupe:n_cupl+n_descl+n_cupe+n_desce]

    #(self, fval, n,M_l, M_e, Spot, flot_l,cal_flot_l, flot_e,cal_flot_e, descu_l, cal_desc_l, 
    #            desc_e, cal_desc_e,plazos, tau,  payer = False, act_360 = True)

    # Valoraciones
    aux = []
    for i in range(len(cup_l)):
      fval_aux = cup_l.index[i]
      flot_l =  cup_l.iloc[i,:].values/100
      flot_l = list(flot_l)
      descu_l = desc_l.iloc[i,:].values/100
      descu_l = list(descu_l)
      cal_flot_l_aux = [fval_aux + timedelta(int(i)) for i in calen_flotante_l]
      cal_desc_l_aux = [fval_aux + timedelta(int(i)) for i in calen_desc_l]

      flot_e =  cup_e.iloc[i,:].values/100
      flot_e = list(flot_e)
      descu_e = desc_e.iloc[i,:].values/100
      descu_e = list(descu_e)
      cal_flot_e_aux = [fval_aux + timedelta(int(i)) for i in calen_flotante_e]
      cal_desc_e_aux = [fval_aux + timedelta(int(i)) for i in calen_desc_e]

      ccs = CrossCurrencySwap()
      ccs.compute(fval_aux, n ,posiciones[0],posiciones[1], X[spot][i], flot_l,cal_flot_l_aux, flot_e,cal_flot_e_aux, descu_l, cal_desc_l_aux,\
                  descu_e, cal_desc_e_aux,plazos, tau)

      aux.append(ccs.precio)

    Y_s['CCS'] = aux

    #print(Y_s)
 
    Y0 = Y_s.head(1)
    Y0 = Y0.values[0]
    #print(Y0)
    PL = Y_s-Y0
    PL['Total']=PL.sum(axis=1)
    #print(PL)
    VaR_C=[]

    if alisado:
      c = 1
      PL.dropna(inplace = True)
      PL = PL.iloc[::-1]
      PL["Esc_SA"] = ExponentialWeighted(0.05,PL.shape[0])
      col_aux = list(PL.columns)
      col_aux.remove("Esc_SA")
      vector_VaR = [VaR_CA(PL.iloc[:,i],PL.Esc_SA,alpha) for i in range(PL.shape[1]-1)]
      VaR = pd.Series(vector_VaR,index = col_aux)
    else:
      c = 0
      VaR=PL.quantile(alpha, numeric_only=False)


    for i in range(0,PL.shape[1]-c):
      VC=CVaR(PL.iloc[:,i],VaR[i])
      VaR_C.append(VC)
    VaR=PL.quantile(alpha, numeric_only=False)
    
    self.summary = pd.DataFrame({"VaR":VaR})
    self.summary['CVaR'] = VaR_C

    self.nocionales = {'Local':posiciones[0],'Extranjero':posiciones[1]}
    self.vencimiento = vencimiento
    self.dias_bono = dias_bono
    self.spot = spot
    self.VaR = self.summary[['VaR']]
    self.CVaR = self.summary[['CVaR']]
    self.alpha = alpha
    self.alisado = alisado
    self.precios = Y0

    return